In [1]:
import re
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).collect()

In [4]:
def hledej(text, n=3, delka=200, frejm=df, sestupne=False, url=True):
    nalezy = frejm.sort(by="datum", descending=sestupne).filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        if url == True:
            print(f'URL: https://www.psp.cz/eknih/{radek[1].replace('_','/')}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [86]:
hledej(r"(?i)[^\w]pizz", sestupne=False)

Kdo: Posl. Sedorjak (malorusky)
Kde: 1925ns_ps_stenprot_150schuz_s150007.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/150schuz/s150007.htm
Kdy: 1928-06-28
Co: st za tento stav má agrární strana. Jest největší vládní stranou, co též úplně vykořisťuje, ale nikoli v zájmu pracujících, nýbrž ve svém vlastním zájmu. Agrárníci přišli na Zakarpatskou Ukrajinu jako Pizzaro do jižní Ameriky. A jako divoká zvěř zuří na polích chudých lidí, zvláště na polích Vrchovinců, tak u nás panují agrárníci všude. Pohlížejí na Podkarpatí jako na velký zbytkový statek, jenž nemůže
***
Kdo: Předseda vlády ČR Miloš Zeman
Kde: 1998ps_stenprot_049schuz_s049109.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/049schuz/s049109.htm
Kdy: 2002-04-25
Co: nci mají právo na obranu jakýmikoliv prostředky. Dovolte mi, abych vám připomněl tuto formulaci: jakýmikoliv prostředky - což v praxi znamená i teroristické bombové útoky na děti, v diskotékách nebo v pizzeriích. Já i v hysterii, která zde ve sněmovně vládla 

In [72]:
hledej(r"[^\w]LSD", sestupne=False)

Kdo: Posl. Božena Procházková
Kde: 1964ns_stenprot_004schuz_s004016.htm
URL: https://www.psp.cz/eknih/1964ns/stenprot/004schuz/s004016.htm
Kdy: 1964-12-10
Co: ě) - objeví v tentýž den, v této době i dvě cedule "Příjem zboží", a to zase ve špičce, kdy je otevřena jediná potravinářská prodejna v obci. Národní výbory jsou v úsilí o nápravu bezmocné. Pracovníci LSD Jednota, okres Břeclav, včetně předsedkyně argumentují, že se nedá nic dělat, že to je nařízeno shora, že inventury budou. Hovory s instruktorem pomohly poodhalit příčinu: jde o šturmování v inventur
***
Kdo: Poslanec Ivan Langer
Kde: 1996ps_stenprot_012schuz_s012051.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/012schuz/s012051.htm
Kdy: 1997-07-08
Co: dy konkrétních a jasných informací a čísel i řadu informací a konstatování, u nichž chybí zdroj či metodika jejich získání. Jako příklad bych uvedl konstatování, že každý experimentátor s halucinogeny LSD přechází na pravidelné užívání, dělení drog na levnější a dražší, poněku

In [70]:
hledej(r"(?i)[^\w]lysohl", sestupne=False)

Kdo: Poslankyně Milada Emmerová
Kde: 1998ps_stenprot_022schuz_s022104.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/022schuz/s022104.htm
Kdy: 2000-02-24
Co: í těchto zdrojů pro výrobu pervitinu přispívá velmi nevhodná televizní reklama. Některé televizní pořady upozorňují dokonce i na jiné využitelné prostředky, jako je například halucinogenní efekt houby lysohlávky kopinaté. Konečně bych chtěla sdělit, že v mnoha státech Evropy zmíněné omezení, týkající se pseudoefedrinu do 30 mg, neexistuje. V podrobné rozpravě bych potom podala pozměňovací návrhy. Děkuji 
***
Kdo: Poslanec Petr Bratský
Kde: 2006ps_stenprot_066schuz_s066262.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/066schuz/s066262.htm
Kdy: 2009-12-10
Co: lezanů, a upřímně řečeno tomuto národnímu sportu holduji například i já osobně, prohledávány za odchodu z lesa košíky, zda v nich náhodou není větší než malé množství nepovolených druhů hub. V případě lysohlávek a podobných houbiček jsou to většinou takové ty travnaté, ř

In [66]:
hledej(r"(?i)[^\w]sluníčkář", sestupne=False)

Kdo: Poslanec Michal Kučera
Kde: 2013ps_stenprot_029schuz_s029187.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/029schuz/s029187.htm
Kdy: 2015-07-07
Co:  navýšení daní. Ano, tato vláda pouze navyšuje daně, zatím jsme nic jiného neviděli. Dovolil bych si nicméně polemizovat s jednou věcí, a to je ten bod, kdy kolega Adamec mluvil o tom, že v uvozovkách sluníčkáři zamítnou herny v jejich obci a mají tzv. nulovou toleranci. Já bych to skutečně nezpochybňoval. V žádném případě nejsem příznivcem nulové tolerance, ale jsem jednoznačně příznivcem toho, aby se obce
***
Kdo: Poslanec Jaroslav Holík
Kde: 2013ps_stenprot_039schuz_s039117.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/039schuz/s039117.htm
Kdy: 2016-01-21
Co: ou promíjeny a budete trestáni stejně jako každý pracující Čech. Nebudeme tolerovat prohlášení: nás si sem pozvala paní Merklová, nám nesmíte nic udělat. Dámy a pánové, já děkuji a jsem zvědavý, kolik sluníčkářů bude za tento návrh napadeno.
***
Kdo: Místopředseda PSP T

In [60]:
hledej(r"(?i)[^\w]mongoloid", sestupne=True)

Kdo: Poslanec Josef Krejsa
Kde: 1996ps_stenprot_010schuz_s010018.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/010schuz/s010018.htm
Kdy: 1997-03-27
Co: ím se s laskavým dovolením pana předsedajícího na okamžik a položím se na otázku, proč název Velký chural. Budu stručný. U poslanců třetí komory půjde, je-li názor předkladatelů správný, jednoznačně o mongoloidní typy. A proč se pro jistotu neobrátím na východ, když přeskočím Rusko. K platům poslanců Velkého churalu: Poslanci naši třetí komory budou mít, protože demokracie, jak tvrdí pan kolega Pithart, je 
***
Kdo: Poslanec Josef Krejsa
Kde: 1996ps_stenprot_003schuz_s003009.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/003schuz/s003009.htm
Kdy: 1996-07-24
Co: cké psychiatrie. Navíc jsem přesvědčen, že tak ryze odborné lékařské výrazy, které jsem na adresu pana ministra v této souvislosti zaslechl, jako - citují s odporem - psychopat, dement, debil, kretén, mongoloidní typ - konec citátu, podle mého tyto odborné výrazy nepatří n

In [62]:
hledej(r"(?i)[^\w]negroid", sestupne=True)

Kdo: Poslanec Pavel Staněk
Kde: 2021ps_stenprot_128schuz_s128083.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/128schuz/s128083.htm
Kdy: 2025-02-12
Co: u", který byl v rámci uvedeného hnutí použit k ovlivnění voličů v rámci předvolební politické kampaně politického hnutí SPD do krajských zastupitelstev v roce 2024, přičemž na plakátu je vyobrazen muž negroidní rasy s agresivním výrazem v obličeji v zakrváceném oblečení a se zakrváceným nožem v ruce, přičemž tento plakát je doplněn textem "Nedostatky ve zdravotnictví nevyřeší ‚chirurgové‘ z dovozu. Stop m
***
Kdo: Poslanec Pavel Staněk
Kde: 2021ps_stenprot_128schuz_s128079.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/128schuz/s128079.htm
Kdy: 2025-02-12
Co: ky uzavřeny - který byl v rámci uvedeného hnutí použit k ovlivnění voličů v rámci předvolební kampaně politického hnutí SPD do krajských zastupitelstev v roce 2024, přičemž na plakátu je vyobrazen muž negroidní rasy s agresivním výrazem v obličeji v zakrváceném oblečení a se

In [52]:
hledej("(?i)cigán", sestupne=True)

Kdo: Poslanec Jaroslav Foldyna
Kde: 2017ps_stenprot_111schuz_s111104.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/111schuz/s111104.htm
Kdy: 2021-07-09
Co: i obyčejní lidé, kteří platí ze svých peněz naši činnost, to znamená tento parlament, co platí ty dávky těch nepřizpůsobivých? (Hovoří velmi emotivně, naléhavě.) Zeptejte se těch slušných kluků, těch cigánů, s kterýma já chodím do hospody. Nevím, kolikrát tady ty humanisti, co tady breptaj o těch věcech, chodí do hospody. Já sedím s tím 
***
Kdo: Místopředseda PSP Jan Bartošek
Kde: 2013ps_stenprot_057schuz_s057289.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/057schuz/s057289.htm
Kdy: 2017-06-06
Co: Děkuji. Dále je to pan prof. Ing. Jindřich Cigánek, CSc.
***
Kdo: Poslanec Petr Lachnit
Kde: 2002ps_stenprot_024schuz_s024021.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/024schuz/s024021.htm
Kdy: 2003-12-05
Co: rních energií, tak jak je to v Dánsku a jinde. A mohl bych pokračovat. Vezměme si osud České energetické agentury,

In [50]:
hledej("(?i)židozedn", sestupne=True)

Kdo: Poslanec Jan Bartošek
Kde: 2017ps_stenprot_002schuz_s002028.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/002schuz/s002028.htm
Kdy: 2017-11-30
Co: at hliníkové čepičky proti chemtrails, a případně další riziko bude to, že Bezpečnostní a informační služba se patrně bude místo vlivu Ruska a Číny mnohem víc zabývat otázkami škodlivosti iluminátů a židozednářů. Z toho důvodu za klub KDU-ČSL říkám, že nepodpoříme tohoto předsedu, protože i při společném vyjednávání jsme deklarovali, že zde nepanuje shoda. To za klub KDU-ČSL.
***
Kdo: Poslanec Pavel Dostál
Kde: 1996ps_stenprot_012schuz_s012080.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/012schuz/s012080.htm
Kdy: 1997-07-10
Co: l "plaváčkem" a dělal si z toho v Republice náramnou legraci. Jindy motivem ke komentáři plného krejsovského černého a nelidského humoru bylo neštěstí vyhořelé rómské rodiny atd. Tuto sněmovnu nazval židozednářským spolkem. My nejsme poslanci voleni v demokratických volbách. Nás sem podle Krejsy dostali 

In [7]:
hledej("(?i)antikonc")

Kdo: Ministr veř. zdravotnictví a tělesné výchovy Šrámek
Kde: 1920ns_ps_stenprot_364schuz_s364002.htm
URL: https://www.psp.cz/eknih/1920ns/ps/stenprot/364schuz/s364002.htm
Kdy: 1925-09-24
Co: gulovati počet porodních pomocnic v určitém okrsku - rayonování - po případě i ustanovovati porodní pomocnice u obcí s určitými fixními platy. Konečně často upozorňují úřady na možnost zákazu prodeje antikoncepčních prostředků a na možnost censury pseudozdravotnické literatury. (Různé výkřiky.) Předseda (zvoní): Žádám o klid.
***
Kdo: Posl. Procházková
Kde: 1964ns_stenprot_018schuz_s018024.htm
URL: https://www.psp.cz/eknih/1964ns/stenprot/018schuz/s018024.htm
Kdy: 1968-01-11
Co: ch školách, v dětských domovech, zanedbávání péče o děti, problémy opuštěných děti a nesnáze s osvojováním, nebo sociálně ekonomické problémy naší mladé generace, populační otázky a s tím související antikoncepce, umělé přerušení těhotenství, pohlavní nemoci a dokonce i protispolečenská činnost, jako je prostituce, a dospě

In [9]:
hledej("(?i)autis[tm]")

Kdo: Poslankyně Monika Horáková
Kde: 1998ps_stenprot_017schuz_s017117.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/017schuz/s017117.htm
Kdy: 1999-10-14
Co:  dalších žáků se speciálními vzdělávacími potřebami, kteří svým inteligenčním kvocientem převyšují uvedenou inteligenční skupinu. Jsou to například žáci postižení souběžně více vadami, žáci postižení autismem, žáci s poruchami učení a chování, žáci ze sociokulturně znevýhodněného prostředí, zejména žáci romského etnika. Děti jsou sice zařazovány do škol na základě odborného psychologického vyšetření ped
***
Kdo: Poslankyně Monika Horáková
Kde: 1998ps_stenprot_017schuz_s017122.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/017schuz/s017122.htm
Kdy: 1999-10-14
Co: li ke konci. Ráda bych měla připomínky jenom k některým vystupujícím v této rozpravě. Na začátku bych ráda uvedla, že pan poslanec Maňásek uváděl, že není pravidlem, že jsou do škol zařazovány děti z autistickými poruchami, děti hyperaktivní a s různými dalšími poruc

In [25]:
hledej("(?i)nagan")

Kdo: Poslanec Michal Kraus
Kde: 1996ps_stenprot_017schuz_s017035.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/017schuz/s017035.htm
Kdy: 1997-12-09
Co: ou prostředky. V roce 1998 se na sportovní reprezentaci počítá s 215 milióny, zatímco v loňském roce to bylo 255 miliónů, a přitom rok 1998 je - jak známo - rokem olympijským a Zimní olympijské hry v Naganu budou finančně velmi náročnou událostí. Na celou oblast investic v oblasti tělovýchovy a sportu zbývá ve státním rozpočtu zhruba 35 miliónů místo po obou balíčcích plánovaných 163, což nestačí ani n
***
Kdo: Poslanec Petr Hulinský
Kde: 1996ps_stenprot_017schuz_s017036.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/017schuz/s017036.htm
Kdy: 1997-12-09
Co: itole VPS. Návrh státního rozpočtu na rok 1998 přiděluje občanským sdružením v tělovýchově a sportu 723 milionů Kč, což představuje pokles o 300 milionů Kč, a to v roce, kdy se koná zimní olympiáda v Naganu. Při projednávání návrhu státního rozpočtu pro rok 1998, kapitola 333 

In [23]:
hledej("(?i)hockey")

Kdo: Posl. Pechmanová
Kde: 1925ns_ps_stenprot_207schuz_s207002.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/207schuz/s207002.htm
Kdy: 1929-06-18
Co: nější, přiznali bychom ji velmi rádi a s velikou radostí - o propagandu našich státních lázní. Bohužel, má s touto propagandou příslovečnou smůlu. Vzpomínám si jen, že na Štrbském plese byly pořádány hockeyové závody. A tu musilo se ministerstvo veř. zdravotnictví se svou propagační pohotovostí dožíti toho zklamání, že těsně před konáním těchto 
***
Kdo: Posl. Brukner
Kde: 1935ns_ps_stenprot_141schuz_s141004.htm
URL: https://www.psp.cz/eknih/1935ns/ps/stenprot/141schuz/s141004.htm
Kdy: 1938-03-17
Co: jež by děti povzbuzovaly. O nic lépe tomu není na měšťance a středních školách. Mám děvče, které chodí na měšťanskou školu, každou chvilku přijde domů s úlohou, má vypracovat slohový úkol o kanadském hockeyi, o Papaninově výpravě na severní točnu, ale nikdy neměla vypracovat úkol o nějaké živnosti nebo živnostníkovi. Vychází spousta ča

In [21]:
hledej("(?i)hokej")

Kdo: Posl. Šalát
Kde: 1935ns_ps_stenprot_076schuz_s076003.htm
URL: https://www.psp.cz/eknih/1935ns/ps/stenprot/076schuz/s076003.htm
Kdy: 1937-01-21
Co: olícku telocvičnú organizáciu orolstva. Nechce byť odvislá od akéhosi byrokratického, diktátorského vlivu brnenského. A hľa, prosím, keby to bol nejaký bezvýznamný spolček "Makkabi" alebo footbalový, hokejový spolok, hneď by to schválili, ale stanovy a žiadosť slovenskej mládeže katolíckej, ktorá je štátu oddaná, ktorá charakterne ukazuje, že chce v rámci štátu pracovať pozitívne na spoločenskom a štát
***
Kdo: Posl. Šafařík
Kde: 1948ns_stenprot_034schuz_s034005.htm
URL: https://www.psp.cz/eknih/1948ns/stenprot/034schuz/s034005.htm
Kdy: 1949-07-14
Co: ho socialistického státu. Naši sokolští sportovci a tělocvikáři zaznamenali od loňského února velké úspěchy na mezinárodním poli, vyvrcholené světovým rekordem Emila Zátopka. Světové prvenství našich hokejistů i Aleny Vrzáňové jsou výmluvnou odpovědí všem nepřátelům a pochybovačům sjednoce

In [11]:
hledej("(?i)ADHD")

Kdo: Poslankyně Jaroslava Wenigerová
Kde: 2010ps_stenprot_033schuz_s033204.htm
URL: https://www.psp.cz/eknih/2010ps/stenprot/033schuz/s033204.htm
Kdy: 2012-02-08
Co: o je zpráva na základě podkladů, tak jestliže v tolika zařízeních 50 % dětí je agresivních vůči pedagogovi, to zřejmě vysvětlí pan kolega Bém, tak se můžeme domnívat, že to mohou být děti například s ADHD a dalšími výchovnými poruchami, které už potřebují péči od mateřské školky. Pak jsou tam další detaily, ale ony to tak úplně detaily nejsou. Pro mě v novele tohoto zákona je velmi nejasné postavení d
***
Kdo: Ministr školství, mládeže a tělovýchovy ČR Josef Dobeš
Kde: 2010ps_stenprot_033schuz_s033207.htm
URL: https://www.psp.cz/eknih/2010ps/stenprot/033schuz/s033207.htm
Kdy: 2012-02-08
Co: om bavit -, si o tom povídat a schválit tady něco, co bude mít většinovou podporu. Přesto si dovolím přijít s malou troškou do mlýna. Já jsem také takové živé dítě jako Pavel Bém a neměl jsem nálepku ADHD/LND, hyperkinetické nesoustředě

In [13]:
hledej("(?i)kokain")

Kdo: Zpravodaj posl. Sedláček
Kde: 1920ns_ps_stenprot_199schuz_s199001.htm
URL: https://www.psp.cz/eknih/1920ns/ps/stenprot/199schuz/s199001.htm
Kdy: 1923-04-10
Co: Slavná sněmovno! Zákonem tímto upraven býti má ve smyslu haagské opiové konvence obchod s opiem, morfiem, kokainem a jinými léky, které vyznačeny jsou jako zdraví lidskému škodlivé. Tento vládní návrh zákona netýká se doposud přímo nás, je to však zákon mezinárodní, a my, nepřijavše ho, tvořili bychom jakousi m
***
Kdo: Zpravodaj posl. Kyncl
Kde: 1925ns_ps_stenprot_152schuz_s152001.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/152schuz/s152001.htm
Kdy: 1928-07-11
Co:  dne 19. února 1925, vyhlášená pod č. 147 Sb. z. a n. z r. 1927. Mezinárodní opiová konvence z 23. ledna 1912 vyplynula z nezbytnosti mezinárodní dohody o postupném potlačování požívání opia, morfia, kokainu, jakož i drog připravených neb odvozených z těchto látek, které dávají neb mohou dáti příležitost k podobnému zneužívání. V důsledku smlouvy versail

In [15]:
hledej("(?i)heroin")

Kdo: Zpravodaj posl. dr Hula
Kde: 1935ns_ps_stenprot_129schuz_s129003.htm
URL: https://www.psp.cz/eknih/1935ns/ps/stenprot/129schuz/s129003.htm
Kdy: 1938-01-27
Co:  dopravováno do přístavních měst a odtud do Ameriky. O velikosti tohoto proudu je možno si učiniti představu z toho, že americké protinarkotikové úřady zabavují ročně cca 13 zásilek narkotik (zejména heroinu a kokainu), které pravidelně dosahují váhy několika metrických centů. Všechny tyto zásilky jsou zadržovány na východním americkém pobřeží, pravidelně ihned po vylodění z lodi, které vypluly z evrops
***
Kdo: Poslanec akademik Charvát
Kde: 1969cnr_stenprot_011schuz_s011004.htm
URL: https://www.psp.cz/eknih/1969cnr/stenprot/011schuz/s011004.htm
Kdy: 1971-03-17
Co: bůstkami, jako jsou cigarety nebo alkohol. I zde však máme do budoucna lepší vyhlídky, než jsou v permisibních populacích, kde se lidé utíkají při řešení svých osobních a interpersonálních problémů k heroinu, marihuaně a jiným drogám. Objevy posledních asi 30 let

In [17]:
hledej("(?i)brno")

Kdo: Poslanec Hampl
Kde: 1918ns_ps_stenprot_005schuz_s005005.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/005schuz/s005005.htm
Kdy: 1918-11-26
Co: t potřebí, abyste si ujasnili situaci, jaká v ohledu zásobovacím panuje zejména v průmyslových centrech. (Hluk.) Jest potřebí přece jenom vžíti se do ovzduší takových velkých měst, jako je na příklad Brno, Praha, Kladno nebo Plzeň, abyste přišli k poznání, že jest potřebí, aby účinná náprava nastala v době pokud možno nejbližší. Jeden z pánů předřečníků - tuším p. koll. Sontág - ve své řeči správně ko
***
Kdo: Poslanec Nohel
Kde: 1918ns_ps_stenprot_006schuz_s006006.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/006schuz/s006006.htm
Kdy: 1918-11-27
Co:  systémem dřívějším a systémem dnešním. V dřívější době bylo Slezsko s přilehlou částí Moravské Ostravy sdruženo v jeden obvod zásobovací tak, že jsme měli vlastně v českých zemích 4 centrály: Prahu, Brno, Opavu a vedle toho Mor. Ostravu. - Mor. Ostrava byla sloučena v jeden obvod

In [8]:
hledej("(?i)internet")

Kdo: Poslankyně Květoslava Čelišová
Kde: 1993ps_stenprot_037schuz_s037019.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/037schuz/s037019.htm
Kdy: 1995-12-07
Co: čnou překážkou zůstane stavební povolení. Opravdu chceme, aby o tak strategicky důležité věci, jakou zcela jistě kabelové rozvody jsou, a nejde pouze o televizní vysílání, ale i o informační dálnici, Internet apod., rozhodoval jeden úředník stavebního odboru obecního úřadu? Jinou věcí je, že v kabelových rozvodech mohou vysílat, resp. vytvářet programy i ti, kteří sami vlastníky rozvodu nejsou. O liberali
***
Kdo: Ministr kultury ČR Jaromír Talíř
Kde: 1996ps_stenprot_010schuz_s010026.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/010schuz/s010026.htm
Kdy: 1997-03-27
Co: najdeme řešení pro jednotlivá zde definovaná témata a pro definované problémové okruhy. Myslím si, že ještě bude nutné, a zapomněl jsem tu uvést, že tato legislativa se také dotkne datových sítí typu Internet a že tady bude třeba také definovat určité ter

In [269]:
hledej(r"(?i)uměl[áéoý]\w{0,7} intelig")

Kdo: Předseda vlády ČR Andrej Babiš
Kde: 2017ps_stenprot_023schuz_s023006.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/023schuz/s023006.htm
Kdy: 2018-11-23
Co: ci, kdy si skutečně můžeme vybírat. Je jasná naše ekonomická strategie. Věda a výzkum a finální výroba. V Bruselu u předsedy Evropské komise jsem lobboval za to, abychom získali centrum excelence pro umělou inteligenci. To je pro nás strašně důležité. Novelu zákona o nemocenském pojištění zavedeme od července příštího roku. Zavedeme elektronickou neschopenku, další důležitý krok k elektronizaci zdravotnictví. T
***
Kdo: Předseda vlády ČR Andrej Babiš
Kde: 2017ps_stenprot_026schuz_s026245.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/026schuz/s026245.htm
Kdy: 2019-01-31
Co: kotvení digitalizace; v prosinci 2018 představení Sonia, bankovní identita pro státní správu, po realizaci získáme kmen 4 milionů klientů do státní správy; v prosinci 2018 schválena strategie využití umělé inteligence v rámci našeho programu Česká rep

In [287]:
hledej(r"(?i)ro\w{0,10} 2025")

Kdo: Poslanec Vladimír Špidla
Kde: 1996ps_stenprot_007schuz_s007040.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/007schuz/s007040.htm
Kdy: 1996-12-18
Co: u uličkou, že musíme uvažovat o tom, že nezaměstnanost může dosáhnout vysloveně katastrofálních hodnot - hovoří se až o 80% nezaměstnanosti, jakkoliv je samozřejmě každá z těchto věcí velmi sporná do roku 2025, že rozhodující pohled je přehodnotit názor na to, co je a co není práce. Vezmeme-li jenom trošku rozšířenou definici, která nebere práci jako něco, za co se dostává mzda, zjistíme, že zhruba 40 % hr
***
Kdo: Poslanec Rostislav Čevela
Kde: 1998ps_stenprot_032schuz_s032033.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/032schuz/s032033.htm
Kdy: 2001-01-24
Co: mu odchodu do důchodu je prospěšné. Stárnutí populace je problém nejen naší země, ale celé Evropy - akcentovaný navíc nízkou porodností a předpokládaným nárůstem důchodců po roce 2005, nemluvě již po roce 2025. A přátelé - zajímejme se o budoucnost, neboť v ní přece 

In [285]:
hledej(r"(?i)me too")

Kdo: Ministr životního prostředí ČR Richard Brabec
Kde: 2017ps_stenprot_040schuz_s040310.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/040schuz/s040310.htm
Kdy: 2020-01-31
Co: jedné konkrétní skládkařské společnosti, která by tady nejradši skládkovala do roku 2150. Ale ono se to naštěstí nepodaří. Tak a teď jenom krátce k některým mýtům. Česká republika... (Reakce z pléna: Me too?) Ne 
***
Kdo: Poslanec Patrik Nacher
Kde: 2017ps_stenprot_100schuz_s100034.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/100schuz/s100034.htm
Kdy: 2021-04-29
Co: okračovat, tak skutečně bude docházet k omezování těch názorů, které nejsou v souladu s tím mainstreamem, s tím jedním názorem. Bavíme se tady o termínech jako fake news, hate speech, cancel culture, Me Too a podobně. A ta základní věc, která mě v tomhle trápí, je, že ti, kteří horují za to mazání, nejsou schopni odlišit jiný názor od lži či projevování nenávisti. Ještě jednou: jiný názor od lži či proj
***
Kdo: Poslankyně Barbora Kořanová
Kd

In [283]:
hledej(r"(?i)cancel cu")

Kdo: Poslanec Patrik Nacher
Kde: 2017ps_stenprot_100schuz_s100034.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/100schuz/s100034.htm
Kdy: 2021-04-29
Co: ou dobu - bude pokračovat, tak skutečně bude docházet k omezování těch názorů, které nejsou v souladu s tím mainstreamem, s tím jedním názorem. Bavíme se tady o termínech jako fake news, hate speech, cancel culture, Me Too a podobně. A ta základní věc, která mě v tomhle trápí, je, že ti, kteří horují za to mazání, nejsou schopni odlišit jiný názor od lži či projevování nenávisti. Ještě jednou: jiný názor o
***
Kdo: Poslanec Andrej Babiš
Kde: 2021ps_stenprot_088schuz_s088025.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/088schuz/s088025.htm
Kdy: 2024-01-17
Co: trování nepohodlných informací nebo umožňovala Evropské komisi zasahovat do ústavně chráněné svobody projevu českých občanů. Abyste se rázně postavil proti šíření atmosféry netolerance, autocenzury a cancel culture. Tento dopis posíláme v kopii poslancům Marku Bendovi, Jan

In [281]:
hledej(r"(?i)geneti")

Kdo: Zpravodaj posl. dr. Uhlíř
Kde: 1918ns_ps_stenprot_112schuz_s112009.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/112schuz/s112009.htm
Kdy: 1920-01-29
Co:  Pansofie neboli vševěda, soustavná sbírka nejdůležitějších poznatků o věcech tohoto světa, 2. Panhistorie, přehled všech vynálezů a zároveň i dějin každé věci a také dějin světových, 3. Pandogmatie, genetický výklad o tom, jak soudili jednotliví mužové o tom a onom vynálezu, až konečně v tom dospěli skutečné pravdy. Proto by bylo podle soudu Komenského potřebí znova se dáti do práce a přezkoumati, jaký
***
Kdo: Posl. inž. dr. Benda
Kde: 1946uns_stenprot_087schuz_s087003.htm
URL: https://www.psp.cz/eknih/1946uns/stenprot/087schuz/s087003.htm
Kdy: 1947-12-16
Co: orné znalosti v té mechanické práci manuální, ale že jim také chybí teoretické znalosti v chovu domácích zvířat. Tito osídlenci nemají dostatečné znalostí o účincích umělých hnojiv, nemají znalosti o genetice domácích zvířat a z toho vyplývající dědičnosti i chovate

In [279]:
hledej(r"DNA")

Kdo: Poslanec Libor Ambrozek
Kde: 1998ps_stenprot_021schuz_s021179.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/021schuz/s021179.htm
Kdy: 2000-01-25
Co:  jakéhokoli nakládání s geneticky modifikovanými organismy. Genové manipulace jsou postupy legitimní, které se v laboratorní praxi využívají od poloviny 70. let. Jejich éra začala s objevem struktury DNA, která je odpovědná za přenos genetické informace v buňce. Nejprve se používaly k čistě teoretickým výzkumům, jak už to v této oblasti bývá, později se objevily obrovské možnosti nejprve v biomedicín
***
Kdo: Poslanec Rostislav Čevela
Kde: 1998ps_stenprot_024schuz_s024089.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/024schuz/s024089.htm
Kdy: 2000-04-06
Co: ogeneze, byl by Merežkovsky zapomenut. Margulisová totiž prokázala, že eukaryotická buňka je v podstatě mnohobuněčný tvor a buněčné organely, jako např. mitochondrie či choroplasty - nositelé vlastní DNA - byly kdysi samostatné organismy, tzv. prokaryonta. Vetřelci a hostit

In [277]:
hledej(r"(?i)trva\w{0,10} udr")

Kdo: Posl. dr. Gustav Heidler
Kde: 1918ns_ps_stenprot_092schuz_s092005.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/092schuz/s092005.htm
Kdy: 1919-11-25
Co: ce, nemůže odpověděti tak určitě, jako odpovídám já osobně, ale i odpověď finančního výboru, pokud je obsažena ve zprávě vám, pánové, předložené, připouští, že politika celní ochrany nebude u nás asi trvale udržitelná. My nemůžeme dnes, jak jsem již řekl, se rozhodnouti a říci: Takovou a takovou obchodní politiku povedeme, ale určitou zásadu musíme i dnes při tomto zmocňovacím zákoně hájiti a proto odhodlal
***
Kdo: Posl. dr. Viškovský
Kde: 1918ns_ps_stenprot_125schuz_s125019.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/125schuz/s125019.htm
Kdy: 1920-02-27
Co: s ohromným obvodem jihočeským má tvořiti nejen ohromnou volební župu, což by nebylo samo o sobě neštěstím - nýbrž také jediný ohromný správní župní obvod, který je téměř monstrem, nezdá se mi býti na trvale udržitelným. Soudím, že čas, zkušenosti a praxe budou

In [42]:
hledej(r"[^\w]Kar\w{0,5} Prager")

Kdo: Poslankyně Věra Kovářová
Kde: 2017ps_stenprot_024schuz_s024057.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/024schuz/s024057.htm
Kdy: 2018-12-05
Co: dpořit tento pozměňovací návrh právě na rehabilitaci památníku Jana Palacha před novou budovu Národního muzea, tedy před někdejším Federálním shromážděním. Památník byl v roce 1969 navržen architektem Karlem Pragerem a měl se sestávat z ocelového pylonu, vysokého téměř 30 metrů, a žulové plastiky Plamen sochaře Miloslava Chlupáče. A právě na onen pylon a umístění kamenné desky by měly být určeny tyto finanční p
***


In [36]:
hledej(r"(?i)[^\w]lesb[iy]")

Kdo: Posl. Krejčí
Kde: 1925ns_ps_stenprot_174schuz_s174006.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/174schuz/s174006.htm
Kdy: 1928-11-08
Co: řítelkyni," a jiný inserát: "Přítele pro celý život hledám". Tedy zde výslovně český tisk strany, která si říká, že je národní, favorisuje a umožňuje protipřírodní pohlavní styk, ať je to t. zv. láska lesbická anebo styk muže s mužem. Jest už jednou na čase, aby také jednou s tohoto místa tomuto svinstvu byla věnována pozornost, a upozornilo se, že takovým způsobem dále jíti nemůžeme. Je zajímavo, co dě
***
Kdo: Poslanec Pavel Dostál
Kde: 1996ps_stenprot_023schuz_s023128.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/023schuz/s023128.htm
Kdy: 1998-03-27
Co: muži. Jenže požadavkem moderní multietnické a multikulturní společnosti je nepochybně, vážení kolegové, tolerance, mimo jiné také tolerance ke vztahu k největší sexuální menšině, homosexuálním mužům a lesbickým ženám. Moderní člověk na homosexualitu nenahlíží jako na zločin, jako

In [34]:
hledej(r"(?i)[^\w]gay[ oie]")

Kdo: Poslanec Pavel Dostál
Kde: 1996ps_stenprot_023schuz_s023128.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/023schuz/s023128.htm
Kdy: 1998-03-27
Co: ejména pak před virem AIDS. Myslím, že trochu práce navíc od státních úředníků za zlepšení vztahu k několika procentům našich spoluobčanů stojí a že se nám všem nakonec jednoznačně vyplatí. Lesbičky a gayové opakovaně žádají, aby stát nějakým způsobem legitimizoval jejich partnerské soužití. Zákon o registrovaném partnerství na tento jejich požadavek reaguje, a proto jej podporuji a doporučuji do druhé
***
Kdo: Poslanec Pavel Maixner
Kde: 1996ps_stenprot_023schuz_s023137.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/023schuz/s023137.htm
Kdy: 1998-03-27
Co: riminaci na základě sexuální orientace - v bydlení, zaměstnání, vzdělání, veřejném ubytování a ve zdravotních a sociálních službách. Schválení dodatku II znamenalo zákaz veškerých zákonů ochraňujících gaye-občany před diskriminací. Nejvyšší soud ve svém rozhodnutí konstatoval:

In [32]:
hledej(r"(?i)[^\w]gay ")

Kdo: Poslanec Zdeněk Krampera
Kde: 1996ps_stenprot_023schuz_s023216.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/023schuz/s023216.htm
Kdy: 1998-04-01
Co: , že homosexuální orientace sama o sobě nesmí být překážkou účasti v řízení o adopci dítěte. Při posledním sčítání lidu a domácností, jež se v Kanadě uskutečnilo dne 14. května 1996, byly spolu žijící gay a lesbické páry poprvé uznány jako samostatná kategorie. Mnoho firem, univerzit a některá města poskytují stejnopohlavním partnerům žijícím ve společné domácnosti s jejich zaměstnanci stejné výhody, j
***
Kdo: Poslanec Jaromír Kohlíček
Kde: 1998ps_stenprot_036schuz_s036122.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/036schuz/s036122.htm
Kdy: 2001-05-17
Co: í neexistuje. Tak se alespoň mimo jiné loni v Chebu vyjádřila. Ptám se vás vzhledem k blížícímu se veřejnému slyšení v Drážďanech v saském Zemském sněmu a k odhalení Policie ČR v posledních měsících - gay kluby v Praze a výroba dětské pornografie, napojená na Belgii: Za 

In [27]:
hledej(r"(?i)[^\w]homosex")

Kdo: Posl. Krejčí
Kde: 1925ns_ps_stenprot_174schuz_s174006.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/174schuz/s174006.htm
Kdy: 1928-11-08
Co:  úkor trvání našeho národa. (Výkřiky.) Slavná sněmovno! Když vezmeme do rukou tyto časopisy, vidíme, že insertní část není nic jiného než bursou prostituce, a nejenom bursou prostituce, nýbrž i bursou homosexuality. (Předsednictví se ujal předseda Malypetr.) "Pražský ilustrovaný zpravodaj" přináší inserát: "Mladá žena hledá přítelkyni," a jiný inserát: "Přítele pro celý život hledám". Tedy zde výslovně če
***
Kdo: Posl. Dvořák
Kde: 1929ns_ps_stenprot_226schuz_s226002.htm
URL: https://www.psp.cz/eknih/1929ns/ps/stenprot/226schuz/s226002.htm
Kdy: 1932-12-13
Co: jovky do Plzně, na zástupce šéfa tohoto oddělení, byl, pánové, generál čsl. armády Mejstřík. (Slyšte!) O této osobě, generálově - zastřeně ovšem - tisk referoval, ovšem ve snaze, aby to svedl na styky homosexuální. Ptám se otevřeně: Jak je to možné, že vojenská prokuratura dosud n

In [275]:
hledej(r"(?i)[^\w]dron")

Kdo: Ministr vnitra ČR Milan Chovanec
Kde: 2013ps_stenprot_023schuz_s023058.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/023schuz/s023058.htm
Kdy: 2014-12-03
Co: voří o tom, že vybuchl další sklad poblíž epicentra původního výbuchu. Letové podmínky v současné době nedovolují, aby Armáda ČR, případně policie vzlétly s helikoptérou a podívaly se nad tento areál. Dron, který tam byl od Armády ČR zapůjčen, byl v minulých dnech stažen a v současné době probíhá jeho přeprava na místo samé. V současné době se tedy letecky tento
***
Kdo: Ministr vnitra ČR Milan Chovanec
Kde: 2013ps_stenprot_023schuz_s023129.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/023schuz/s023129.htm
Kdy: 2014-12-04
Co: ky de facto na perimetru daného areálu se zvedl a viděl, že tam hoří jiná budova. Já jsem dnes požádal pana kolegu Stropnického a budu ho žádat i v rámci předsednictva Bezpečnostní rady státu, aby ten dron, který byl stažen, zůstal na místě do té doby, než bude veškerá munice odvezena, protože je

In [273]:
hledej(r"(?i)kyberbezp")

Kdo: Poslanec Ivan Gabal
Kde: 2013ps_stenprot_053schuz_s053016.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/053schuz/s053016.htm
Kdy: 2016-11-29
Co: ských služeb na novou úroveň. Samozřejmě ještě je poměrně dost práce, která nám chybí. Ale je důležité, abychom si uvědomili, že ze všech koutů tohoto parlamentu děláme významný krok, který se netýká kyberbezpečnosti nebo dalších věcí, ale který se týká kvalitativně nového stupně zpravodajských služeb. Tolik na faktické upřesnění. A já bych byl rád a poděkoval bych všem, kteří se o tohle zasloužili. Doufám
***
Kdo: Poslanec Zbyněk Stanjura
Kde: 2013ps_stenprot_053schuz_s053061.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/053schuz/s053061.htm
Kdy: 2016-11-30
Co: y přednesla návrh dopoledne, podrobně ho zdůvodnila, o navýšení prostředků na obranu ve výši 10 miliard. Jenom připomínám, jedna miliarda byla určena pro Vojenské zpravodajství, zejména pro zajištění kyberbezpečnosti, a 9 miliard bylo směřováno na samotnou Armádu České rep

In [271]:
hledej(r"(?i)kyberúto")

Kdo: Předseda vlády ČR Bohuslav Sobotka
Kde: 2013ps_stenprot_048schuz_s048119.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/048schuz/s048119.htm
Kdy: 2016-06-30
Co: žádal paní poslankyni, aby mě upozornila na nějaké konkrétní případy, kde si myslí, že by mohlo dojít k nějakému ohrožení bezpečnosti informačních systémů České republiky, kde tady jsou nějaká rizika kyberútoků nebo kyberšpionáže. Takže znovu opakuji, abych mohl posoudit relevanci těch informací a těch rešerší, které zde citovala paní poslankyně Němcová, musel bych mít širší znalost o tom, o jaké přesně p
***
Kdo: Předseda vlády ČR Bohuslav Sobotka
Kde: 2013ps_stenprot_054schuz_s054113.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/054schuz/s054113.htm
Kdy: 2017-01-12
Co: izik směřujeme k připravenosti České republiky na rizika, která patří k moderní informační společnosti. Koneckonců i audit národní bezpečnosti, který jsme realizovali, jasně avizoval, že právě otázka kyberútoků je něco, čemu vláda musí věnovat zvýše

In [10]:
hledej(r"(?i)bitcoin")

Kdo: Poslanec Zbyněk Stanjura
Kde: 2013ps_stenprot_033schuz_s033113.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/033schuz/s033113.htm
Kdy: 2015-10-07
Co: sely. A je to nevhodný nástroj a my se snažíme vysvětlit, proč je nevhodný. Já si nechám, řekl bych, možná ty lehčí otázky, na to třetí čtení. Například by mě zajímalo, v kolika restauracích se platí bitcoiny, když myslíme na 
***
Kdo: Poslanec Adolf Beznoska
Kde: 2013ps_stenprot_047schuz_s047158.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/047schuz/s047158.htm
Kdy: 2016-05-31
Co: v i na nás, na běžné občany. Částka 10 tisíc eur se může zdát relativně vysoká, ale předesílám, že některé evropské země tuto částku již mají poloviční. Návrh začal řešit i virtuální měny, jako třeba bitcoiny, a zahrnuje osoby poskytující služby spojené s virtuální měnou do okruhu povinných osob, kterým vyplývají povinnosti vyplývající z tohoto zákona. To samozřejmě může snížit jistou atraktivitu tohoto 
***
Kdo: Předseda vlády ČR Andrej Babiš
Kde:

In [11]:
hledej(r"(?i)www")

Kdo: Místopředseda vlády Pavel Mertlík
Kde: 1998ps_stenprot_013schuz_s013242.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/013schuz/s013242.htm
Kdy: 1999-05-20
Co:  tím, co obsahuje hospodářská část zprávy o stavu společnosti. Myslím, že nemá cenu vypočítávat to, co je ve zprávě řečeno. Zpráva byla včas publikována jak na internetu, na vládním serveru na adrese www.vlada.cz, tak v Hospodářských novinách, což je všeobecně čtený a respektovaný deník. Sněmovna ji samozřejmě dostala včas. Chtěl bych říci, že dnes prožíváme hospodářskou recesi, která po roce 1990 ne
***
Kdo: Ministr bez portfeje ČR Karel Březina
Kde: 1998ps_stenprot_036schuz_s036252.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/036schuz/s036252.htm
Kdy: 2001-05-24
Co: m na starosti oblasti informačních a komunikačních technologií, tak všechny dokumenty, které vláda v této oblasti přijímá, jsou přístupné na internetu, a to jak na internetových stránkách Úřadu vlády www.vlada.cz, anebo na internetových stránkách Úřadu

In [12]:
hledej(r"(?i)faceboo")

Kdo: Poslanec Ondřej Liška
Kde: 2006ps_stenprot_059schuz_s059199.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/059schuz/s059199.htm
Kdy: 2009-06-17
Co: otvrzeno či vyvráceno, zda maturita je, nebo není dobře připravena. To znamená aby byl důsledně hájen zájem těch, kteří budou první generací, která bude skládat státní maturitní zkoušku. To, že se na facebooku přihlásilo několik desítek tisíc mladých lidí, kteří mají strach z něčeho nového, o čem přesně všechno nemohou vědět, protože je to velmi složitý proces, do kterého vidí skutečně jen několik desíte
***
Kdo: Poslankyně Olga Zubová
Kde: 2006ps_stenprot_060schuz_s060071.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/060schuz/s060071.htm
Kdy: 2009-09-09
Co: louchat jim. Státní maturitu odmítá stále víc lidí. Svůj nesouhlas vyjádřilo přes 20 tisíc lidí v petici za odklad státních maturit. Bezmála 60 tisíc členů obsahuje každá ze dvou skupin sociální sítě Facebook, které vyjadřují svůj nesouhlas s projektem, který jste vy, poslan

In [13]:
hledej(r"(?i)whatsapp")

Kdo: Poslanec Marian Jurečka
Kde: 2017ps_stenprot_026schuz_s026307.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/026schuz/s026307.htm
Kdy: 2019-02-12
Co: ci předtím s migranty ani s žádným ilegálním, řekněme, lídrem nějaké skupiny, který by to organizoval, v kontaktu nebyl. Byl v kontaktu pouze s rodinou. Dokonce se prověřovala třeba i komunikace přes WhatsApp. Také tam není to, co by tam mělo být, a to znamená, že by mu za tu službu bylo zaplaceno buď předem, nebo poté. Takže to jsou silné argumenty pro to, abychom se mohli domnívat, že to řízení nebylo o
***
Kdo: Poslanec Ondřej Profant
Kde: 2017ps_stenprot_062schuz_s062340.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/062schuz/s062340.htm
Kdy: 2020-11-12
Co: ernet. Proto máme v Ústavě právo na soukromí. Proto mě velmi rmoutí, že na výboru pro evropské záležitosti 4. listopadu byla vzata na vědomí výjimka z ePrivacy směrnice, aby poskytovatelé služeb typu WhatsApp a podobně prolamovali šifrování. To je zcela anachronické, jde

In [14]:
hledej(r"(?i)tiktok")

Kdo: Ministr školství, mládeže a tělovýchovy ČR Vladimír Balaš
Kde: 2021ps_stenprot_035schuz_s035119.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/035schuz/s035119.htm
Kdy: 2022-09-08
Co: tak se snažíme navázat kontakt se všemi potenciálními ukrajinskými asistenty nebo pedagogy. Problém byl v tom, že oni používají jiné sociální sítě, než na jaké my jsme zvyklí, například Telegram nebo TikTok. Ale máme i přehled o tom, v jakých krajích a v jakých ORP ten problém může být, je to spíše v těch větších aglomeracích, jak jste zmínil. A máme samozřejmě tendenci také s nimi spolupracovat a zvyšo
***
Kdo: Ministr školství, mládeže a tělovýchovy ČR Vladimír Balaš
Kde: 2021ps_stenprot_035schuz_s035124.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/035schuz/s035124.htm
Kdy: 2022-09-08
Co:  situace je velmi pestrá. My se snažíme zjistit další informace. Já jsem říkal, že jeden problém je v tom, že fungují Ukrajinci na jiných sociálních sítích, než my jsme zvyklí, je to hlavně Telegram, TikTo

In [15]:
hledej(r"(?i)fake news")

Kdo: Ministr zdravotnictví ČR Miloslav Ludvík
Kde: 2013ps_stenprot_058schuz_s058020.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/058schuz/s058020.htm
Kdy: 2017-05-10
Co: e překvapovat, jakým způsobem se v téhle zemi vytvářejí nebo vyrábějí kauzy. Z tohoto se udělá kauza. Finále je co? Nic. Kdybych byl blonďák a seděl jsem o něco západněji, tak bych to nazval typickou fake news. Ale já touto cestou cítím, a omlouvám se, že tady zdržuji, cítím trochu povinnost se za prvé obhájit a za druhé říci, že tenhle způsob obchodování s informacemi je strašný, a skoro je mi líto, že to
***
Kdo: Poslankyně Helena Langšádlová
Kde: 2013ps_stenprot_057schuz_s057371.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/057schuz/s057371.htm
Kdy: 2017-06-08
Co:  Koudelka na serveru HlídacíPes. Z rozhovoru vyplývají mnohé negativní skutečnosti. Jedná se o nárůst webových stránek, na kterých se velmi často setkáváme s proruskou propagandou, ale i s tím, že se fake news vyskytují i ve zdánlivě seriózních mé

In [16]:
hledej(r"(?i)hoax")

Kdo: Poslanec Jan Klán
Kde: 2013ps_stenprot_055schuz_s055275.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/055schuz/s055275.htm
Kdy: 2017-03-03
Co:  Aldoux Huxley a jeho kniha Konec civilizace, která říká, že budeme žít v takové změti různých informací a budeme v tom mít takový zmatek, že nebudeme vědět, co je pravda a co ne. A to souvisí s těmi hoaxy a faky, že je takové množství dat, že nevíme, které je pravdivé. Takže elektronické občanské průkazy mají spojitost s kontrolovanou společností, protože někdo někde bude o mně mít další dodatečnou k
***
Kdo: Místopředseda vlády ČR Pavel Bělobrádek
Kde: 2013ps_stenprot_060schuz_s060003.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/060schuz/s060003.htm
Kdy: 2017-09-05
Co: e, anebo skutečně používá fake news, jak naši europoslanci hlasovali ohledně uvalení sankcí na Českou republiku za nepřijímání migrantů. Prostřednictvím řetězových e-mailů se mezi českými občany šíří hoax o tom, jak nám čeští vlastizrádci z Evropského parlamentu hl

In [17]:
hledej(r"(?i)dezinf")

Kdo: Prezident Gottwald pri tejto príležitosti povedal
Kde: 1948ns_stenprot_040schuz_s040003.htm
URL: https://www.psp.cz/eknih/1948ns/stenprot/040schuz/s040003.htm
Kdy: 1950-02-22
Co: italistických štátoch, si nijako vojnu nepraje. A to je príčina, prečo podnecovatelia novej vojny vrhajú sa naň s obrovským aparátom. "V Spojených štátoch amerických vládne prekvapujúci stav verejnej dezinformovanosti", povedal raz prezident Roosevelt. A s týmto stavom zámerne udržovanej 
***
Kdo: Předsedkyně Sněmovny lidu MUDr. S. Pennigerová
Kde: 1969fs_sl_stenprot_004schuz_s004002.htm
URL: https://www.psp.cz/eknih/1969fs/sl/stenprot/004schuz/s004002.htm
Kdy: 1969-10-15
Co: í mnozí pocit, že byli oklamáni. Avšak nevidí ještě dosud skutečné viníky, a to ty, kteří s nimi kalkulovali a v období vypjatých politických situací je měli připravené jako želízka v ohni. Politická dezinformace je u mládeže zvlášť silná. Je u ní i silná deziluze. Chybí jí dnes vzory, ideály, někdo nebo něco, za čím by mohla jít a č

In [18]:
hledej(r"(?i)[^\w]hovn[aoye]\w{0,1}[^\w]")

Kdo: Poslanec Jan Volný
Kde: 2021ps_stenprot_027schuz_s027055.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/027schuz/s027055.htm
Kdy: 2022-06-23
Co: hodil se nám sem zpovídat ten ředitel. To byl poctivý a neskutečně pracovitý kluk, holt možná někde víc agilní, než bylo zdrávo. To je pravda. Ale tady to vytahovat a říkat, vy jste to dělali úplně na hovno, my to budeme dělat líp? No, je to politika. Děkuji za slovo.
***
Kdo: Poslanec Roman Kubíček
Kde: 2021ps_stenprot_115schuz_s115127.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/115schuz/s115127.htm
Kdy: 2024-10-23
Co: , tak to prostě je. Máte jezdit do toho Bruselu a máte říkat, že to nemá být klasický finanční derivát. Vy máte říkat, že si to mají vybírat ti eminenti. Když s tím nic neděláte, tak jste tam úplně na hovno! Pardon.
***


In [19]:
hledej(r"(?i)zakroutiti krk")

Kdo: Posl. dr Koberg (německy)
Kde: 1925ns_ps_stenprot_041schuz_s041016.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/041schuz/s041016.htm
Kdy: 1926-06-25
Co: si mají pomoci. Již se stanoviska většiny mělo se uniknouti byť jen zdání nenávisti, aby se nepřišlo do podezření, že se zde provozuje čistý protekční systém, a že jest úmysl těm, kteří se znelíbili, zakroutiti krk. A když se tyto výjevy odehrávaly, dostala se vůbec republika do nebezpečí, že dlužno sáhnouti k zákonu na ochranu republiky? Domnívám se, že se republika těmito výjevy naprosto nezakolísala a že dne
***
Kdo: Posl. Gottwald (pokračuje)
Kde: 1929ns_ps_stenprot_007schuz_s007003.htm
URL: https://www.psp.cz/eknih/1929ns/ps/stenprot/007schuz/s007003.htm
Kdy: 1929-12-21
Co:  jsme stranou československého proletariátu a naším nejvyšším revolučním štábem je skutečně Moskva. A my se chodíme do Moskvy učit, víte co? My se od ruských bolševiků do Moskvy chodíme učit, jak vám zakroutiti krk. (Výkřiky. - Veselost.) A vy víte

In [20]:
hledej(r"(?i)Petr Pavel")

Kdo: Poslanec Petr Hulinský
Kde: 2010ps_stenprot_040schuz_s040166.htm
URL: https://www.psp.cz/eknih/2010ps/stenprot/040schuz/s040166.htm
Kdy: 2012-06-12
Co: k stojí před úkolem adaptovat se co nejdříve na nové potřeby a cíle bezpečnostní a branné politiky NATO a Evropské unie. Věřím, že k tomu může přispět i nový náčelník Generálního štábu generálporučík Petr Pavel. Jedno je ale v případě Afghánistánu pro armádu velmi důležité. Naši vojáci obstáli v dlouholeté afghánské misi se ctí, a proto je třeba našim vojákům za vzornou reprezentaci České republiky poděkova
***
Kdo: Poslankyně Jana Černochová
Kde: 2013ps_stenprot_057schuz_s057039.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/057schuz/s057039.htm
Kdy: 2017-05-17
Co: icie, ze kterého během dvou týdnů z podplukovníka udělal generála. Mohl si vybrat svého ředitele Vojenského zpravodajství. Mohl si vybrat svého náčelníka Generálního štábu, protože končil pan generál Petr Pavel a novým náčelníkem Generálního štábu se stal pan generál

In [21]:
hledej(r"(?i)koronavir")

Kdo: Poslanec Bohuslav Svoboda
Kde: 2017ps_stenprot_040schuz_s040160.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/040schuz/s040160.htm
Kdy: 2020-01-28
Co: Já to zkusím znovu. Navrhuji zařazení bodu na pořad dnešního jednání jako první bod Informace vlády ČR k opatřením souvisejícím se šířením koronaviru. Už jsem několikrát řekl, není mým cílem vyvolávat paniku, to v žádném případě, ale Sněmovna i lidé v České republice by měli dostat ujištění, že je vláda schopna reagovat, pokud se věci nebudou vyv
***
Kdo: Předseda PSP Radek Vondráček
Kde: 2017ps_stenprot_040schuz_s040162.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/040schuz/s040162.htm
Kdy: 2020-01-28
Co: návrh nebyl přijat. Pak je zde návrh pana poslance Bohuslava Svobody, abychom do pořadu schůze zařadili nový bod, kterým je Informace vlády, případně ministra zdravotnictví, o opatřeních proti šíření koronaviru. Zahajuji hlasování. Ptám se, kdo je pro zařazení nového bodu. Kdo je proti? V hlasování číslo 115 přihlášeno 165

In [22]:
hledej(r"(?i)modulá\w{0,10} reakto")

Kdo: Poslanec Petr Třešňák
Kde: 2017ps_stenprot_051schuz_s051042.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/051schuz/s051042.htm
Kdy: 2020-06-18
Co: i, resp. usnesení paní Balcarové, které by nově znělo: "aby výrazně podpořila výzkum, vývoj a inovace v oblasti obnovitelných zdrojů energie, včetně akumulace energie z těchto zdrojů, a výzkum malých modulárních reaktorů a jejich využití v našich podmínkách." Předám písemně.
***
Kdo: Poslankyně Dana Balcarová
Kde: 2017ps_stenprot_051schuz_s051042.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/051schuz/s051042.htm
Kdy: 2020-06-18
Co:  Teď čtu vlastně ten alternativní návrh pana kolegy Třešňáka: "aby výrazně podpořila výzkum, vývoj a inovace v oblasti obnovitelných zdrojů energie, včetně akumulace energie z těchto zdrojů, a malých modulárních reaktorů a jejich využití v našich přírodních podmínkách".
***
Kdo: Poslanec Petr Třešňák
Kde: 2017ps_stenprot_058schuz_s058185.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/058schuz/s0581

In [23]:
hledej(r"(?i)tokamak")

Kdo: Poslanec prof. ing. Ladislav Kubíček, DrSc.
Kde: 1981cnr_stenprot_002schuz_s002006.htm
URL: https://www.psp.cz/eknih/1981cnr/stenprot/002schuz/s002006.htm
Kdy: 1981-07-06
Co: technickému zvládnutí termojaderné energie jako nového zdroje energie. Československá účast na tomto programu bude soustředěna na řešení dodatečného ohřevu plazmatu vysokofrekvenčními poli v zařízení Tokamak, na řešení problémů laserového ohřevu plazmatu apod. Významná a exponovaná účast našich vědců bude dále zaměřena na integrační opatření RVHP týkající se nových způsobů využití sluneční energie a dalš
***
Kdo: Ministryně pro vědu a výzkum ČR Helena Langšádlová
Kde: 2021ps_stenprot_050schuz_s050096.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/050schuz/s050096.htm
Kdy: 2023-01-12
Co: du, a to o 850 milionů. Zaměřili jsme se na ty nejpalčivější oblasti, a to je na posílení DKRVO pro vysoké školy o 450 milionů, navýšení DKRVO pro Akademie věd o 200 milionů a na financování projektu Tokamak o 200 milionů, 

In [24]:
hledej(r"(?i)(twitter|tweet)")

Kdo: Předseda vlády ČR Petr Nečas
Kde: 2010ps_stenprot_033schuz_s033230.htm
URL: https://www.psp.cz/eknih/2010ps/stenprot/033schuz/s033230.htm
Kdy: 2012-02-09
Co: orupčních opatření. Součástí webových stránek budou i modelové příklady s dopady reformních kroků. Od počátku roku propagujeme tento web v rámci avíz a tiskových zpráv na sociálních sítích Facebook a TwitterTwitter
***
Kdo: Poslanec Adolf Beznoska
Kde: 2013ps_stenprot_010schuz_s010124.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/010schuz/s010124.htm
Kdy: 2014-06-13
Co: ckých dat, ale po tomhle projevu přece jenom aspoň jedna věta. Jsem dalek toho, abych někomu říkal, jak mají vypadat projevy ve Sněmovně, jakou mají mít úroveň. Proto si dovolím volně odcitovat jeden twittertwitter
***
Kdo: Poslanec Miroslav Kalousek
Kde: 2013ps_stenprot_020schuz_s020047.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/020schuz/s020047.htm
Kdy: 2014-11-07
Co: Zatímco jsme tady, pane premiére, rokovali o vládních návrzích z dílny pana mini

In [25]:
hledej(r"HIV")

Kdo: Ministr zdravotnictví ČR Martin Bojar
Kde: 1990cnr_stenprot_021schuz_s021034.htm
URL: https://www.psp.cz/eknih/1990cnr/stenprot/021schuz/s021034.htm
Kdy: 1991-07-11
Co: ibulkovi. Poslední poznámka. Nezbývá ještě, než užít jeden argument: Uvědomme si, že ať chceme nebo nechceme, během dvou let bude aspoň Česká republika vystavena velkému problému séropozitivity virem HIV u skupiny lidí, kteří představují rizikovou skupinu, jsou to toxikomani, lidé, kteří podléhají různým kombinovaným závislostem a je prokazatelné, že u těchto skupin populace je desetkrát až patnáctkr
***
Kdo: Poslanec Jaroslav Štrait
Kde: 1993ps_stenprot_014schuz_s014023.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/014schuz/s014023.htm
Kdy: 1993-11-11
Co:  mezi prohlášením tiskové mluvčí paní Soukupové a experta Ministerstva zdravotnictví pana Petra Turka. Každý hovoří o jiných firmách, které figurují v současném vyšetřování preparátů nakažených virem HIV, které nyní probíhá v Německu. Chtěl bych slyšet ujiště

In [26]:
hledej(r"(?i)tunelář")

Kdo: Poslanec Josef Krejsa
Kde: 1996ps_stenprot_011schuz_s011023.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/011schuz/s011023.htm
Kdy: 1997-05-15
Co: o být umožnění porovnat daňová přiznání občanů s jejich nově nabytým majetkem či penězi na bankovních kontech. Tím lze okamžitě zjistit, kdo majetek získal legálně a kdo ho má jako podíl či pozornost tunelářské mafie. Nemohu se zbavit dojmu, že právě mnou uvedené si vláda nepřeje, a proto zákon, jak je navržen, vůbec srovnání placení daní a nabytého majetku neumožňuje. Jediným zářným příkladem tak parado
***
Kdo: Poslanec Pavel Dostál
Kde: 1996ps_stenprot_011schuz_s011048.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/011schuz/s011048.htm
Kdy: 1997-05-21
Co: etodou a o tunelování, které provádí banky, nad kterými vaše centrální banka má provádět dozor. V tisku byly rozebírány různé aféry v návaznosti na kupónovou privatizaci. Bohužel obvykle pozdě, neboť tuneláři pracují bez reklamy a rychle. Zatím se hovořilo o vytunelování bank

In [27]:
hledej(r"(?i)hermelín")

Kdo: Posl. dr Mareš
Kde: 1929ns_ps_stenprot_033schuz_s033004.htm
URL: https://www.psp.cz/eknih/1929ns/ps/stenprot/033schuz/s033004.htm
Kdy: 1930-03-21
Co: eálem ohromná kasárna, ve kterých by se tísnili lidé v malých bytech o 40 m2 nebo dokonce v obytných kuchyních. Situace je však taková, že musíme zatím řešiti, co nás nejvíce tíží. Mrznu-li, nehledám hermelín, vezmu cokoliv. Naším ideálem musí býti dostati lidi z těch tmavých a vlhkých činžáků, ze starých nevlídných domů o 6, 7 poschodích. Chceme členy našeho národa vyvésti blíže k půdě, do rodinných domk
***
Kdo: Poslankyně V. Kuželová
Kde: 1976fs_sl_stenprot_003schuz_s003011.htm
URL: https://www.psp.cz/eknih/1976fs/sl/stenprot/003schuz/s003011.htm
Kdy: 1978-10-24
Co: navštívila některé potravinářské závody v našem okrese Příbram a ráda bych se s vámi podělila o zkušenosti, které jsem získala. Jako první jsem navštívila LAKTOS Sedlčany, známý svým výrobkem - sýrem Hermelín - který za pomoci Výzkumného ústavu mlékárenského vyrobil sý

In [28]:
hledej(r"(?i)sex\w{0,10} hara")

Kdo: Poslanec Josef Jalůvka
Kde: 1998ps_stenprot_024schuz_s024155.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/024schuz/s024155.htm
Kdy: 2000-04-07
Co: ucí chování sexuální povahy je neakceptovatelné a může být za jistých okolností v rozporu s čl. 3 a 5 Směrnice Rady č. 76/207 EHS. Doporučení Komise č. 92/131 ES definuje pojem "sexuální obtěžování", sexual harassment, a podporuje členské státy v rozšiřování kodexu praktik boje proti sexuálnímu obtěžování ve veřejném sektoru - dále jen kodex boje. Dalším dokumentem je Deklarace Rady z 19. prosince 1991 o imp
***
Kdo: Poslanec Jiří Bláha
Kde: 2017ps_stenprot_007schuz_s007125.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/007schuz/s007125.htm
Kdy: 2018-03-02
Co: ceme hodnotit, oprostit se od minulosti. (Nesrozumitelné výkřiky z pravé části sálu.) A to bychom také mohli dojít k názoru, že všichni pánové, kteří jsou tady dneska, vlastně by mohli být nařčeni ze sexuálního harašení, že osahávali v době, kdy byli v pubertě, své spoluža

In [29]:
hledej(r"(?i)menstru")

Kdo: Posl. Krejčí
Kde: 1925ns_ps_stenprot_174schuz_s174006.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/174schuz/s174006.htm
Kdy: 1928-11-08
Co: le hrajeme na schovávanou. Člověk nemusí býti ani lékařem a přece ví, že porucha měsíčků v 99% je znamením těhotenství. Jestliže se doporučují vyzývavým dryáčnickým způsobem prostředky, kterými by se menstruace obnovila, znamená to zabití plodu, třeba byl teprve před měsícem nebo dvěma počat. Tu jsme na ohromném omylu. Paní kol. Karpíšková říká, že já věci nerozumím. Dovolil bych si jí říci, že ona a sna
***
Kdo: Posl. Hatina
Kde: 1929ns_ps_stenprot_189schuz_s189002.htm
URL: https://www.psp.cz/eknih/1929ns/ps/stenprot/189schuz/s189002.htm
Kdy: 1932-05-25
Co: terá velmi těžce a ve velkém množství postihuje dělnice zaměstnané v tabákových továrnách, choroba, která přináší dělnicím v tabákových továrnách veliké útrapy. Dělnice trpí chudokrevností, sešlostí, menstruačními poruchami a velmi často dostavují se předčasné potraty. Toto ustanov

In [30]:
hledej(r"(?i)menstru\w{0,10} chudoba")

Kdo: Místopředsedkyně PSP Klára Dostálová
Kde: 2021ps_stenprot_075schuz_s075003.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/075schuz/s075003.htm
Kdy: 2023-09-07
Co: árokovat, nebo se snažit přijít s pozměňovacími návrhy, které sníží daňové sazby hygienických a menstruačních pomůcek. I vy jste dostali, všichni jsme dostali do mailu stanovisko odborné, že skutečně menstruační chudoba je realitou pro tisíce žen, a já nechápu, proč by měly být tyto pomůcky ve vyšší sazbě daně než například už několikrát zmiňované pornočasopisy. Obdobně je to i s květinami. Já vím, že to vyvolá úsmě
***
Kdo: Poslankyně Ivana Mádlová
Kde: 2021ps_stenprot_075schuz_s075039.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/075schuz/s075039.htm
Kdy: 2023-09-07
Co: o děti. Takže to ano, to ten důvod samozřejmě je. Jinak co se týče těch sociálně slabých, které mají velké problémy s tím, aby si obstaraly dámské hygienické potřeby. Dneska už je celkem vžitý termín menstruační chudoba. Ano, zaznamenala jsem i r

In [31]:
hledej(r"(?i)změn\w{0,10} pohlaví")

Kdo: Ministryně zdravotnictví ČR Zuzana Roithová
Kde: 1996ps_stenprot_020schuz_s020149.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/020schuz/s020149.htm
Kdy: 1998-02-10
Co: mu podstatnější změny. 1. Zákon upravuje některé výkony a postupy, které zásadním způsobem ovlivňují integritu jedince, to je darování orgánů, odběry tkání, pitvy, asistovaná reprodukce, sterilizace, změna pohlaví transsexuálních osob, kastrace sexuálních delikventů. Tyto postupy jsou dosud regulovány pouze podzákonnými normami. 2. V oblasti úpravy povinností poskytovatelů zdravotní péče a zdravotnických praco
***
Kdo: Ministr zdravotnictví ČR Leoš Heger
Kde: 2010ps_stenprot_020schuz_s020023.htm
URL: https://www.psp.cz/eknih/2010ps/stenprot/020schuz/s020023.htm
Kdy: 2011-07-12
Co: jejichž součástí je provádění specifických zdravotních výkonů, které mohou podstatným způsobem ovlivnit život nebo zdraví pacienta. Jedná se konkrétně o asistovanou reprodukci, sterilizaci, kastraci, změnu pohlaví, psychochirurgické v

In [32]:
hledej(r"(?i)osobn\w{0,10} počíta")

Kdo: Ministr elektrotechnického průmyslu ČSSR M. Kubát
Kde: 1981fs_sl_stenprot_003schuz_s003015.htm
URL: https://www.psp.cz/eknih/1981fs/sl/stenprot/003schuz/s003015.htm
Kdy: 1983-10-11
Co: R a ve druhé variantě v kooperaci spolu s firmou Philips - Rakousko. Rozbíhá se výroba nové minivěže, což je tuner, přehrávač, zesilovač v jedné sestavě spolu s malými reproduktory. Rozbíhá se výroba osobních počítačů, i když musím přiznat, že ceny zatím ne zcela odpovídají doložce "osobní". Rozběhla se výroba celé řady typů elektronických hodin a hodinek v kooperaci se SSSR. Rozbíhá se dále výroba většího počtu
***
Kdo: Poslanec Petr Koháček
Kde: 1993ps_stenprot_029schuz_s029009.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/029schuz/s029009.htm
Kdy: 1995-03-15
Co: ést odpovědnost i za text předložený na technickém nosiči dat. Toto rozšíření je zároveň otevřením cesty k realizaci usnesení § 120, odst. 3. Vzhledem k tomu, že všichni poslanci mají k dispozici své osobní počítače, je to věc, která 

In [33]:
hledej(r"(?i)mobil\w{0,10} telef")

Kdo: Místopředseda PSP Jan Kasal
Kde: 1993ps_stenprot_039schuz_s039066.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/039schuz/s039066.htm
Kdy: 1996-02-21
Co: ti 21. Konstatuji, že jsme přijali usnesení, kterým jsme zamítli návrh poslanců Jiřího Hájka a dalších. Nyní se budeme zabývat  O úvodní slovo žádám pověřeného člena vlády, ale pan ministr Dlouhý nám mobilním telefonem sdělil, že se intenzívně pokouší dostat do budovy Poslanecké sněmovny, takže musíme chvilku vyčkat. Nic jiného nám nezbývá. Pan poslanec Štambera má náhradní kartu č. 12, pan poslanec Opatřil má 
***
Kdo: Místopředseda PSP Jiří Vlach
Kde: 1996ps_stenprot_007schuz_s007032.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/007schuz/s007032.htm
Kdy: 1996-12-13
Co: to tematicky provázáno. To je zatím nepoužitý a velmi zvláštní postup. Pan kolega Filip se hlásí. Dámy a pánové, zvažte, jestli nepoužijete dobrý zvyk z lepších anglických restaurací, že se odkládají mobilní telefony v šatně. (Potlesk.) Když ne, tak se asp

In [34]:
hledej(r"(?i)selfie")

Kdo: Poslanec Petr Bendl
Kde: 2013ps_stenprot_036schuz_s036332.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/036schuz/s036332.htm
Kdy: 2015-12-16
Co: Tak, já to nebudu radši komentovat. Že by nějaké další foto? Nějaké selfie že bychom tam zase šoupli místo projednávaného návrhu na rozpočtové určení daní. (Veselost ve sněmovně.) Já myslím, že se musíme vrátit k tomu začátku, proč vlastně vůbec tento návrh zákona projednáv
***
Kdo: Poslankyně Monika Jarošová
Kde: 2017ps_stenprot_040schuz_s040440.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/040schuz/s040440.htm
Kdy: 2020-02-13
Co:  týká obrany českých zájmů, jinak jste hovořil před našimi občany, ale zcela opačně, v rozporu s vašimi slovy, hlasovali vaši poslanci v europarlamentu. Jen připomenu, vy si fotíte s Viktorem Orbánem selfie, a vaši poslanci pak hlasují pro rezoluci proti Maďarsku. Vy hájíte na domácí scéně V4, a komisařka Jourová vyhrožuje Polsku v souvislosti s reformou justice. Uvědomujete si vůbec zhoubnost toho, kdy

In [35]:
hledej(r"(?i)sex\w{0,10} obtěž")

Kdo: Poslanec Jindřich Nehera
Kde: 1996ps_stenprot_023schuz_s023225.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/023schuz/s023225.htm
Kdy: 1998-04-02
Co: ého vyznání, rasy, etnického původu či příslušnosti ke kočovným společenstvím, jako jsou mj. irští cikání, dráteníci, příslušníci ekologických a duchovních společenství, a jenž by postavil mimo zákon sexuální obtěžování v zaměstnání a při přípravě na zaměstnání. Irský parlament měl projednat návrh zákona o rovnosti v zaměstnání na podzim roku 1996. Návrh zákona o rovnosti v zaměstnání rozšiřuje zákon z roku 199
***
Kdo: Poslanec Josef Jalůvka
Kde: 1998ps_stenprot_024schuz_s024155.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/024schuz/s024155.htm
Kdy: 2000-04-07
Co: ovisko Parlamentního institutu. Cituji: Orgány Evropské unie opakovaně, a jistě oprávněně, v řadě dokumentů poukazují na nutnost aktivního postupu členských států proti snižování lidské důstojnosti a sexuálnímu obtěžování na pracovištích. Problematika důstojnosti a

In [36]:
hledej(r"politic\w{0,10} korekt")

Kdo: Poslanec P. Kanis
Kde: 1990fs_sn_stenprot_001schuz_s001003.htm
URL: https://www.psp.cz/eknih/1990fs/sn/stenprot/001schuz/s001003.htm
Kdy: 1990-06-27
Co: cie, aké miesta dostanú jednotlivé politické strany a hnutia. Druhý krok pri tvorbe týchto orgánov boli vlastné návrhy politických orgánov, strán a hnutí, kto koho bude zastupovať. Zdá sa mi, že táto politická korektnosť a politická kultúra tvorby orgánov Federálneho zhromaždenia získava slabé miesto práve návrhom nového bodu 3. Pôvodný bod 3, ktorý si máte možnosť prečítať, končí vetou "každý poslanec môže predn
***
Kdo: Poslanec Jiří Müller
Kde: 1990cnr_stenprot_013schuz_s013008.htm
URL: https://www.psp.cz/eknih/1990cnr/stenprot/013schuz/s013008.htm
Kdy: 1991-01-17
Co: nesmí mít charakter politického nástroje jedněch proti druhým. To je naprosto základní věc a je třeba ji ctít. Byl bych rád, kdyby se ČNR podařilo přispět k tomu, aby byl dán průchod pravdě, věcným a politicky korektním způsobem. Děkuji. (Potlesk.)
***
Kdo: Poslan

In [37]:
hledej(r"influence")

Kdo: Poslanec Dominik Feri
Kde: 2017ps_stenprot_062schuz_s062109.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/062schuz/s062109.htm
Kdy: 2020-10-22
Co: rnetu, ohledně komunikace nějakých opatření, zejména tedy těch, která jsou výkladově složitější. Tím myslím, jestli jste například zvažovali nějakou mediální kampaň, jestli jste zvažovali kampaň přes influencery v tom nejširším slova smyslu, to znamená, ať už jsou to celebrity, nebo ať už jsou to 
***
Kdo: Ministr zdravotnictví ČR Roman Prymula
Kde: 2017ps_stenprot_062schuz_s062110.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/062schuz/s062110.htm
Kdy: 2020-10-22
Co: způsobem, protože tady to nebezpečí je zcela zřejmé a my jsme opravdu v situaci, kdy velká část národa nevěří tomu, co se komunikuje, a proto je nezbytné to vysvětlovat nepochybně lépe a i za využití influencerů, jak tady padlo. To znamená, tuto strategii chceme začít realizovat opravdu rychle. Když mám hovořit o komunikaci s Ministerstvem zdravotnictví, tak ta osob

In [38]:
hledej(r"hybridn\w{0,10} vál")

Kdo: Poslanec Bohuslav Chalupa
Kde: 2013ps_stenprot_044schuz_s044037.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/044schuz/s044037.htm
Kdy: 2016-04-13
Co: jenom v souvislosti s tím, co se tady hovoří dneska a řeší - internet a svoboda. Chci upozornit na to, že v podstatě nemáme ani vyřešeno to, jak se budeme chovat na internetu, když se v podstatě vede hybridní válka, nebezpečí terorismu a podobné věci. Takže pokud se tady budeme bavit o internetu a omezování svobody, tak vyzývám i kolegy z opozice, ať se zapojí do diskuse na výboru pro obranu a pro bezpečnost 
***
Kdo: Poslanec Bohuslav Chalupa
Kde: 2013ps_stenprot_057schuz_s057239.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/057schuz/s057239.htm
Kdy: 2017-05-25
Co: ny, které v tuto chvíli i díky tomu, že jsme z různých důvodů desítky let upozaďovali význam kybernetického prostoru jako prostoru, kde bude docházet, a již evidentně dochází, k hybridním kampaním, k hybridním válkám, a tedy ke střetům, které mají nepřátelský cha

In [39]:
hledej(r"hybridn\w{0,10} hroz")

Kdo: Předseda vlády ČR Bohuslav Sobotka
Kde: 2013ps_stenprot_047schuz_s047238.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/047schuz/s047238.htm
Kdy: 2016-06-02
Co: y měly být finalizovány další kapitoly auditu. Patří mezi ně hrozby v kyberprostoru, energetická, surovinová a průmyslová bezpečnost, otázka působení cizí moci, otázka organizovaného zločinu a otázka hybridních hrozeb. A také nezávislá kapitola, která je zpracovávána Českou národní bankou, stabilita měny a finančních institucí. Opět nejprve, až budeme mít výsledky těchto kapitol, proběhne konference na půdě Posl
***
Kdo: Poslankyně Dana Váhalová
Kde: 2013ps_stenprot_047schuz_s047238.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/047schuz/s047238.htm
Kdy: 2016-06-02
Co: Děkuji za odpověď, pane premiére. Jak vím, audit už nám teď odhalil, že je nutné, aby stát měl centrum, které bude komplexně monitorovat teroristické a hybridní hrozby i tzv. propagandu cizí moci. Gestorem by mělo být Ministerstvo vnitra. Já bych se je

In [40]:
hledej(r"přiroz\w{0,10} porod")

Kdo: Poslankyně Taťána Fischerová
Kde: 2002ps_stenprot_023schuz_s023158.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/023schuz/s023158.htm
Kdy: 2003-12-02
Co: . K tomuto poznání už došlo v zahraničí mnoho odborníků i rodičů a to právě vedlo k posunu myšlení a ke vzniku porodních domů a center, ve kterých je porod veden pouze porodními asistentkami, protože přirozený porod přece není nemoc, avšak v případě komplikací je tu samozřejmě smluvní i praktická návaznost na lékařskou péči. Diskuse o porodních asistentkách se nespravedlivě stáčí k problematice domácích porodů, 
***


In [41]:
hledej(r"humanright")

Kdo: Poslanec Svatopluk Karásek
Kde: 2002ps_stenprot_045schuz_s045074.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/045schuz/s045074.htm
Kdy: 2005-06-16
Co: Vážený pane místopředsedo, členové vlády, kolegyně a kolegové, předem bych chtěl ujistit paní ministryni, paní předkladatelku, že beru vážně varování našeho ctěného pana prezidenta, který varuje před humanrightismem, to znamená, že ani já nechci "rajtovat" na lidských právech, že si vážím práce Ministerstva školství a ústavů v úsilí o humanizaci ústavní výchovy, s tím, že musí dbát hlavně o zdraví a život s
***
Kdo: Ministryně školství, mládeže a tělovýchovy ČR Petra Buzková
Kde: 2002ps_stenprot_045schuz_s045080.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/045schuz/s045080.htm
Kdy: 2005-06-16
Co: ásek. Mě mrzí, že tady pan poslanec Karásek není, ale nevadí. Já bych ho chtěla ujistit, že Ministerstvo školství ani já osobně si opravdu, ale opravdu neděláme nic z varování prezidenta Klause proti humanrightistům. My se 
***
Kd

In [42]:
hledej(r"lidsk\w{0,10} práv")

Kdo: Posl. dr. Herben
Kde: 1918ns_ps_stenprot_031schuz_s031002.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/031schuz/s031002.htm
Kdy: 1919-02-20
Co:  I. 7. srpna 1873: "Každý, kdo přijal křest, náleží nějakým způsobem papeži." Toto učení je tak obecné, že paderbornský biskup Martin ve svém spisu ("Biskupské slovo") řekl: "Já jsem podle božského i lidského práva pravým vrchním pastýřem také protestantů ve své diecési." Právo církve, upalovat kacíře, nebylo dosud nikdy v církvi odvoláno a katolický biskup Hefele v r. 1870 doznal: "Hierarchii opravdu nechybí 
***
Kdo: Zpravodaj posl. Houser
Kde: 1918ns_ps_stenprot_044schuz_s044003.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/044schuz/s044003.htm
Kdy: 1919-04-10
Co: sem slyšel, je ten, že odporuje náš návrh zákona zákonu rakouskému z roku 1883. Ano, odporuje. Chceme, aby ten rakouský zákon byl odstraněn. Školský zákon, zvaný novela, byla zrovna urážka základních lidských práv, které dříve Rakousko prohlásilo a která, doufám

In [43]:
hledej(r"vegan")

Kdo: Poslanec Jiří Valenta
Kde: 2013ps_stenprot_050schuz_s050287.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/050schuz/s050287.htm
Kdy: 2016-11-08
Co: rmy Norek s. r. o. Dovolím si zmínit alespoň tři hlavní mýty. Za prvé: Důvodová zpráva je ve svých částech neobjektivní a nevyvážená. Argumentace v ní přesně kopírují argumenty organizací s radikální veganskou vizí, což je tedy velice úsměvné, přičemž absolutně nebere do úvahy námitky přímo dotknutého sektoru a nereflektuje existující vědecké důkazy. Za druhé: Vědecká literatura domestikaci druhů chova
***
Kdo: Poslanec Jiří Dolejš
Kde: 2013ps_stenprot_050schuz_s050291.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/050schuz/s050291.htm
Kdy: 2016-11-08
Co: lství české či jiné. Samozřejmě že chovatelé budou vesele chovat dál, že se po přijetí tohoto zákona nedočkáme toho, že budou zavírány masny, že každý z nás bude nucen ke coming outu a prohlásí se za vegana, ale že jde o to, že se nemají chovat konkrétní zvířata, tedy šelmy. A n

In [44]:
hledej(r"pražs\w{0,10} okruh")

Kdo: Místopředseda Petr
Kde: 1946uns_stenprot_083schuz_s083008.htm
URL: https://www.psp.cz/eknih/1946uns/stenprot/083schuz/s083008.htm
Kdy: 1947-11-25
Co: Žádám p. posl. Horu, aby přednesl svůj dotaz na pana ministra techniky ve věci stavby dálnice Praha - Humpolec a pražského okruhu.
***
Kdo: Posl. Hora (čte)
Kde: 1946uns_stenprot_083schuz_s083008.htm
URL: https://www.psp.cz/eknih/1946uns/stenprot/083schuz/s083008.htm
Kdy: 1947-11-25
Co: Pane předsedo, slavná sněmovno, paní a pánové! Činím stručný dotaz podle jedn. řádu na pana ministra techniky ve věci stavby dálnice Praha - Humpolec a pražského okruhu. Poloha Československé republiky může pro nás znamenat mnohé výhody soustředění komunikačních spojů a jich vedení přes naše uzemí. Od roku 1939 bylo plánováno a započato se stavbou úseku dálnice Pra
***
Kdo: K dotazu pana posl. Hory dovoluji si odpověděti takto
Kde: 1946uns_stenprot_083schuz_s083008.htm
URL: https://www.psp.cz/eknih/1946uns/stenprot/083schuz/s083008.htm
Kdy: 1947-11-25

In [45]:
hledej(r"NATO")

Kdo: Predseda vlády Viliam Široký (uvítaný potleskom)
Kde: 1954ns_stenprot_006schuz_s006002.htm
URL: https://www.psp.cz/eknih/1954ns/stenprot/006schuz/s006002.htm
Kdy: 1955-05-24
Co:  a niekedy i výslovne ťuzákonenomŤ vmiešovaní sa do vnútorných vecí ťzmluvnýchŤ partnerov. Práve v tom je hlavná vnútorná slabosť takých imperialistických paktov, ako je takzvaná Západoeurópska únia, NATO, SEATO a podobne. Zmluva o priateľstve, spolupráci a vzájomnej pomoci je jednak novým utužením a prehĺbením našich existujúcich spojeneckých zväzkov, jednak ich rozšírením na ďalšie spriatelené kraji
***
Kdo: Zpravodaj posl. Hruška
Kde: 1954ns_stenprot_016schuz_s016013.htm
URL: https://www.psp.cz/eknih/1954ns/stenprot/016schuz/s016013.htm
Kdy: 1957-03-06
Co: bujeme, neboť západní Německo přikročuje k vybudování armády, která má být hrozbou druhým národům Evropy, hlavně ve východní Evropě. Vždyť hitlerovský generál je již jmenován velitelem pozemní armády NATO, která nemá absolutně mírové snahy, ale má sku

In [46]:
hledej(r"(?i)Don Pabl")

Kdo: Poslanec Pavel Blažek
Kde: 2017ps_stenprot_020schuz_s020004.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/020schuz/s020004.htm
Kdy: 2018-10-23
Co: světlím, proč jsem to tak pojmenoval a o čem by to bylo, a předpokládám, že minimálně s kolegy z hnutí ANO se shodnu na tom, že tento bod zařadíme a to usnesení schválíme. Ten bod by se jmenoval "Lex Don Pablo Brunensis et Augustus Andreas Ciconia." Proč se prosím tento bod má takto nazývat, je jednoduché. Já si tak sedím pěkně doma v Brně a najednou se ze všech novin patřících Agrofertu na mě hrne nějaký
***
Kdo: Poslanec Pavel Blažek
Kde: 2017ps_stenprot_020schuz_s020005.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/020schuz/s020005.htm
Kdy: 2018-10-23
Co: ě mým hlavním tehdejším politickým odpůrcům též z Brna. A tehdy Svatopluk Bartík, a já myslím, že geniálně a pohotově, vymyslel tuhle přiléhavou přezdívku. Tak se s tím musím naučit žít. Když už jsem Don Pablo, tak uznáte - a poprosím příště pana předsedajícího, aby si necha

In [47]:
hledej(r"(?i)Žít Brn")

Kdo: Poslanec Pavel Blažek
Kde: 2017ps_stenprot_020schuz_s020004.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/020schuz/s020004.htm
Kdy: 2018-10-23
Co: a, ale jaký já jsem teda strašný člověk a říká se mi Don Pablo. Tak abychom si tady zase rozuměli, tak tu přezdívku Don Pablo, ona skutečně existuje, vím o ní, vymyslel jistý Svatopluk Bartík z hnutí Žít Brno. To je tentýž Svatopluk Bartík, který se s prezidentem republiky soudí o to, jestli prezident má nebo nemá rakovinu, a vymyslel to zase v období pražských brnologů a brnomytologů.
***
Kdo: Poslankyně Veronika Vrecionová
Kde: 2017ps_stenprot_027schuz_s027380.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/027schuz/s027380.htm
Kdy: 2019-03-27
Co: , mužem, který je zřejmě - promiňte mi ta slova, ale vypůjčím si slovník pana premiéra - hlavou mafiánské chobotnice, která v minulých letech na radnici Brno-střed ovládala v době vlády koalice ANO a Žít Brno tuto městskou část a podílela se zřejmě na rozsáhlé trestné činnosti. Pan Šva

In [48]:
hledej(r"(?i)sado-maso")

Kdo: Poslanec Andrej Babiš
Kde: 2021ps_stenprot_074schuz_s074032.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/074schuz/s074032.htm
Kdy: 2023-09-05
Co: ošek z KDU-ČSL, a mluvil o našich dětech. Ano, drogy pro mě jsou svinstvo, a já nevím, co to bylo tam za lajnu nebo nebylo. Ale já jsem se dozvěděl, Matěj Holáň, Žít Brno - Alenka tu není, to bylo to sado-maso - ne, to je ten z toho Brna. A co nám říká pan Holáň, Žít Brno? Kritika je pokrytecká zejména od politiků, užívání nelegálních návykových látek je v politice běžné. Chápete to? Užívá někdo drogy tady
***


In [49]:
hledej(r"(?i)sado\w{0,1}maso")

Kdo: Poslanec Martin Komárek
Kde: 2013ps_stenprot_058schuz_s058014.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/058schuz/s058014.htm
Kdy: 2017-05-10
Co: ět na to, o čem mluvila paní kolegyně Pekarová. Ona mluvila o tom, které praktiky jsou nám milé a které jsou přípustné. Nuže já mám jednoduchou odpověď. To, co předvádí teď tato Sněmovna, je brutální sadomaso. Děkuji.
***
Kdo: Místopředseda vlády a ministr vnitra ČR Jan Hamáček
Kde: 2017ps_stenprot_076schuz_s076015.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/076schuz/s076015.htm
Kdy: 2020-12-09
Co: vané nouzové stavy někdy do půlky jara příštího roku. Nikdo to neřeší, protože prostě vědí, že to bez toho nouzového stavu nezvládnou. Akorát my máme to české specifikum, že si to tady každých 14 dní sadomasochisticky zopakujeme, a ty lidi na to koukají a nevěří svým vlastním očím. Já vím, že to nezměním, protože je to ideální prostor k prezentaci, je to ideální prostor k tomu, jak tady vládu v tom koupat
***
Kdo: Poslanec Lubomír

In [50]:
hledej(r"(?i)tunelova")

Kdo: Poslanec Vojtěch Filip
Kde: 1996ps_stenprot_008schuz_s008033.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/008schuz/s008033.htm
Kdy: 1997-02-11
Co: slu a nebyla to Komunistická strana Čech a Moravy, kdo způsobila nesoběstačnost země v takových zemědělských produktech, jako je obilí a brambory. Nebyla to ani Komunistická strana Čech a Moravy, kdo tuneloval banky a mohl bych pokračovat dál ve věcech všeobecně známých. (V sále hluk, předsedající žádá klid.) Je vidět, že některé věci se v této sněmovně nedobře poslouchají. Dočkali jsme se označení "lůza"
***
Kdo: Poslanec Jaroslav Gongol
Kde: 1996ps_stenprot_008schuz_s008108.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/008schuz/s008108.htm
Kdy: 1997-02-27
Co:  léta vůbec nebyl uzavřen audit, neboť auditor v roce 1994 a 1995 nedohledal majetek asi za 30 miliard korun. Věřím, že i pro vás je částka 30 miliard korun nezanedbatelná. I když samozřejmě částky vytunelovaných bank jsou zřejmě vyšší. Nevede to k myšlence, že když se n

In [51]:
hledej(r"(?i)ekoterori")

Kdo: Poslanec Oldřich Vojíř
Kde: 1998ps_stenprot_037schuz_s037060.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/037schuz/s037060.htm
Kdy: 2001-06-27
Co:  spíše už měly být pak už jen čistě technického rázu, a nikoli takového širokého rázu, který u nás je někdy praktikován, a dochází potom k tomu, že - a promiňte ten možná trochu tvrdý výraz - někteří ekoteroristé v podstatě pak využijí toho, že mohou zablokovat některé činnosti bez ohledu na to, zdali jsou, nebo nejsou bezpečné, protože si to pouze přejí a mají jinou ideu, jak k některým věcem přistupovat,
***
Kdo: Poslanec Martin Říman
Kde: 2002ps_stenprot_030schuz_s030049.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/030schuz/s030049.htm
Kdy: 2004-03-24
Co: vátní příslušnou náhradu, ať už ve formě pozemku, nebo ve formě obydlí nebo jiné stavby, pak samozřejmě tyto problémy padnou. Bude jich daleko méně. Pak opravdu budeme moci hovořit o tom, že takzvaní ekoteroristé se spolčili proti tu D8, tu D3, a už to nebude problém normá

In [52]:
hledej(r"(?i)klimatic\w{0,10} kriz")

Kdo: Místopředseda PSP Tomáš Hanzel
Kde: 2017ps_stenprot_027schuz_s027345.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/027schuz/s027345.htm
Kdy: 2019-03-27
Co:  Dříve či později stejně přestaneme těžit a spalovat uhlí, musíme ale vytvořit nové pracovní příležitosti. Lidé musí mít nejdřív na chleba, a pak až dokážou přemýšlet o změně klimatu. Je třeba využít klimatickou krizi jako příležitost pro modernizaci Moravskoslezského a Ústeckého kraje. Investovat do rozvoje nových nízkoemisních technologií výroby. Začít skutečně efektivně podporovat obnovitelné zdroje, ne tu vyt
***
Kdo: Poslanec Václav Klaus
Kde: 2017ps_stenprot_039schuz_s039342.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/039schuz/s039342.htm
Kdy: 2019-12-06
Co: lení matematický nebo základní logický nějaký divný. Argumenty matematika se špatně učí, tak počkáme do roku 2090, až se bude učit dobře - to také není řešení, to už přece budeme dávno mrtví kvůli té klimatické krizi (ozývá se smích), čili to není řešení, kt

In [53]:
hledej(r"(?i)obnovit\w{0,10} zdroj")

Kdo: Místopředseda vlády ČSSR L. Gerle
Kde: 1981fs_sl_stenprot_002schuz_s002003.htm
URL: https://www.psp.cz/eknih/1981fs/sl/stenprot/002schuz/s002003.htm
Kdy: 1982-10-20
Co: zační opatření jsou zaměřena na využití odpadního tepla z kompresních stanic tranzitního plynovodu pro zemědělsko-potravinářské komplexy, na inovaci horkovzdušných sušáren a na využívání druhotných a obnovitelných zdrojů energie, např. sluneční. Podstatný přínos má být dosažen modernizací zemědělské automobilové dopravy při současném snížení podílu traktorové dopravy. Při sestavování návrhu plánu na rok 1983 zjišťuj
***
Kdo: Předseda Československé akademie věd akademik Bohumil Kvasil
Kde: 1981cnr_stenprot_006schuz_s006010.htm
URL: https://www.psp.cz/eknih/1981cnr/stenprot/006schuz/s006010.htm
Kdy: 1982-11-09
Co: ých technologií a bioinženýrství v zemědělství, lékařství i průmyslu. Vedle těchto ohnisek a spolu s nimi razí si cestu nové postupy v energetice a zhodnocování surovin /jako jsou jaderná energetika, obnov

In [54]:
hledej(r"(?i)větrn\w{0,10} elekt")

Kdo: Poslanec Jaroslav Žižka
Kde: 1992cnr_stenprot_007schuz_s007018.htm
URL: https://www.psp.cz/eknih/1992cnr/stenprot/007schuz/s007018.htm
Kdy: 1992-11-20
Co: lachovací systémy. Dále 84 81 80 319 termostatické ventily topných radiátorů. Další položka: 84 21 39 716 automobilové katalyzátory. Položka 85 39 39 102 úsporné světelné zdroje. Položka 85 02 30 919 větrné elektrárny do výkonu 0,5 MW a malé vodní elektrárny do výkonu 1 MW. Poslední položka 84 19 19 009 solární zařízení. Vážení kolegové a kolegyně, prosím, aby jednotlivé položky byly považovány za samostatné 
***
Kdo: Poslanec Vladimír Budinský
Kde: 1992cnr_stenprot_007schuz_s007018.htm
URL: https://www.psp.cz/eknih/1992cnr/stenprot/007schuz/s007018.htm
Kdy: 1992-11-20
Co: y. A toto se nám stane - vzpomeňme na záležitost, kterou jsme před chvílí absolvovali s těmi pozměňovacími návrhy na daň z příjmu - to se nám může stát i v dalších bodech. Stejně tak můžeme hovořit o větrných elektrárnách nebo malých vodních elektrárnách, vždy

In [55]:
hledej(r"(?i)recykl")

Kdo: Předseda Československé akademie zemědělské akademik Karel Kudrna
Kde: 1981cnr_stenprot_007schuz_s007011.htm
URL: https://www.psp.cz/eknih/1981cnr/stenprot/007schuz/s007011.htm
Kdy: 1982-12-21
Co: 1. Vytvoření takové struktury ZS, jež s maximální přesností odpovídá podmínkám konzervativních rysů krajinného prostoru republiky. 2. Transformace a utilizace všech hmot, jež odcházejí ze ZS a jejich recyklace. Přitom je potřeba zdůraznit, že ZS je bezprostředně závislá na vlastnostech krajinného prostoru a že svou činností jej také nejvíce přetváří; spolu s lesním hospodářstvím pracujeme na 91 % plochy
***
Kdo: Ministr průmyslu ČSR Miroslav Kapoun, dipl. tech.
Kde: 1981cnr_stenprot_012schuz_s012010.htm
URL: https://www.psp.cz/eknih/1981cnr/stenprot/012schuz/s012010.htm
Kdy: 1984-04-03
Co: eskoslovenska o efektivním využívání druhotných surovin a odpadů v národním hospodářství. Vláda tím zároveň plní závazky obsažené v deklaraci a maloodpadové a bezodpadové technologii a znovuvyužití a r

In [56]:
hledej(r"(?i)sociál\w{0,10} sí[tť]")

Kdo: Ministr práce a sociálních věcí Milan Horálek
Kde: 1990cnr_stenprot_012schuz_s012018.htm
URL: https://www.psp.cz/eknih/1990cnr/stenprot/012schuz/s012018.htm
Kdy: 1990-12-19
Co: te, že je ani nemáme. Máme 7 miliard na valorizaci důchodů. Důchody u nás činí 70, proto to je 10%. A takto se také bavíme o mzdách. Byl bych rád, abyste věděli, když jde o sociální smír, o vytvoření sociální sítě pro tento klid, pro tento hlavní předpoklad naší reformy, tak se musíme také dívat na vybudování této sítě, na naše kompetence, na naši činnost, protože i v oblasti zaměstnanosti je toto nesmírně dů
***
Kdo: Poslanec Andrej Gjurič
Kde: 1990cnr_stenprot_012schuz_s012019.htm
URL: https://www.psp.cz/eknih/1990cnr/stenprot/012schuz/s012019.htm
Kdy: 1990-12-19
Co:  lidmi. Ztrátou zaměstnání přichází člověk o velikou část svých významných mezilidských vztahů. Bývá však druhá, zpravidla ještě podstatnější část, a tou je rodina. Ta představuje skutečně záchrannou sociální síť nezaměstnaného. Kde tato síť 

In [57]:
hledej(r"(?i)bitcoin")

Kdo: Poslanec Zbyněk Stanjura
Kde: 2013ps_stenprot_033schuz_s033113.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/033schuz/s033113.htm
Kdy: 2015-10-07
Co: sely. A je to nevhodný nástroj a my se snažíme vysvětlit, proč je nevhodný. Já si nechám, řekl bych, možná ty lehčí otázky, na to třetí čtení. Například by mě zajímalo, v kolika restauracích se platí bitcoiny, když myslíme na 
***
Kdo: Poslanec Adolf Beznoska
Kde: 2013ps_stenprot_047schuz_s047158.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/047schuz/s047158.htm
Kdy: 2016-05-31
Co: v i na nás, na běžné občany. Částka 10 tisíc eur se může zdát relativně vysoká, ale předesílám, že některé evropské země tuto částku již mají poloviční. Návrh začal řešit i virtuální měny, jako třeba bitcoiny, a zahrnuje osoby poskytující služby spojené s virtuální měnou do okruhu povinných osob, kterým vyplývají povinnosti vyplývající z tohoto zákona. To samozřejmě může snížit jistou atraktivitu tohoto 
***
Kdo: Předseda vlády ČR Andrej Babiš
Kde:

In [58]:
hledej(r"(?i)czexit")

Kdo: Poslanec Martin Lank
Kde: 2013ps_stenprot_047schuz_s047006.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/047schuz/s047006.htm
Kdy: 2016-05-24
Co:  kde variantu odchodu z Evropské unie položili v podobě referenda skutečně na stůl a jednoznačně na tom získávají. Důkazem je to, že si Británie už teď vyjednala podstatné ústupky. Na téma takzvaného czexitu napsal velmi zajímavou analýzu makroekonom Pavel Kohout, mimo jiné dvě volební období byl členem Národní ekonomické rady vlády. Vyšel z toho, že se osobně ve Velké Británii zúčastnil simulace jednán
***
Kdo: Poslanec Bohuslav Chalupa
Kde: 2013ps_stenprot_048schuz_s048139.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/048schuz/s048139.htm
Kdy: 2016-06-30
Co:  na výboru pro obranu, toho, co se dozvídám na podvýboru pro imigraci a azylovou politiku, na evropských záležitostech. A to, že jsem řekl brexit, tak to je příměr. Protože samozřejmě můžeme mluvit o czexitu, o polském exitu a tak dále. Jestli se budeme bavit tou formou, ja

In [59]:
hledej(r"(?i)schengen")

Kdo: Ministr vnitra ČR Jan Ruml
Kde: 1993ps_stenprot_025schuz_s025063.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/025schuz/s025063.htm
Kdy: 1994-12-15
Co: ké skupiny v Záhřebu v roce 1995. Tato skupina je tvořena z expertů následujících vlád: Maďarsko, které předsedá této komisi, Česká republika, Turecko, Chorvatsko, Evropská unie zastupovaná Německem, Schengenská skupina zastupovaná Německem. Práce této skupiny bude podporována třemi mezinárodními institucemi, Mezinárodní organizací pro migraci, Mezivládní konzultací a Mezinárodním centrem pro vývoj migrač
***
Kdo: Poslanec Jaroslav Štrait
Kde: 1996ps_stenprot_017schuz_s017019.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/017schuz/s017019.htm
Kdy: 1997-12-04
Co: nchronizaci naší zahraniční politiky v zásadních otázkách a rozhodnutích při vstupu do nadnárodních útvarů. Nedovedu si dobře představit, že na hranicích česko-slovenských začnou platit v budoucnosti Schengenské dohody. Nedovedu si také představit, že si budeme deset

In [60]:
hledej(r"(?i)multikul")

Kdo: Poslanec Petr Pospíšil
Kde: 1990cnr_stenprot_024schuz_s024063.htm
URL: https://www.psp.cz/eknih/1990cnr/stenprot/024schuz/s024063.htm
Kdy: 1991-11-15
Co: matu. Tak je nyní předkládána jako teoretické východisko k diskusi. Konkretizace a podrobnější rozpracování je záležitostí několika let. Dále se ministerstvo odvolává na vzdálenou budoucnost "v rámci multikulturní Evropy" ale není jasné, jaká je a bude nejlepší perspektiva výchovy. Zpětná vazba, která může ovlivnit například skladby předmětů samostatným rozhodováním ředitelů při přidělování časové dotace 
***
Kdo: Prezident České republiky Václav Havel
Kde: 1993ps_stenprot_006schuz_s006003.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/006schuz/s006003.htm
Kdy: 1993-02-23
Co: l prostor, v němž žijeme, stále těsnější, což vyvolává stále větší vzájemnou nesnášenlivost všech, kteří ho obývají. Možná, že v kotli tohoto celosvětového zápasu se zvolna a těžce rodí nová éra. Éra multikulturního světa jako světa, který je schopen vstř

In [61]:
hledej(r"(?i)gender")

Kdo: Tam legionář V. Vandas - o jeho pravdomluvnosti bude zajisté něco věděti pan kol. Jaša - psal o generálovi Gajdovi takto
Kde: 1920ns_ps_stenprot_306schuz_s306009.htm
URL: https://www.psp.cz/eknih/1920ns/ps/stenprot/306schuz/s306009.htm
Kdy: 1924-11-26
Co: u ke generálovi tomskou buržoasií. Vůbec celý portrét činil dojem symbolického zobrazení daru, krvavého daru ruské revoluce od cizozemské reakce. Já chápal jsem carského hrdinu Rusko-japonské války Megenderfa, vezoucího za sebou vagon dojných krav, ale proč vagon portrétů?"
***
Kdo: Poslankyně Květoslava Čelišová
Kde: 2002ps_stenprot_041schuz_s041092.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/041schuz/s041092.htm
Kdy: 2005-02-10
Co: án, se týká problematiky menšin. Nebudu zde hovořit o romských menšinách a dalších. Daleko problematičtější situace - a jsem o tom přesvědčena nejenom ze svých zkušeností a znalostí, ale z kontaktu s gender studies a s různými neziskovými a nevládními ženskými organizacemi - je v diskriminaci ž

In [62]:
hledej(r"AIDS")

Kdo: Předseda ČNR Josef Kempný
Kde: 1986cnr_stenprot_009schuz_s009011.htm
URL: https://www.psp.cz/eknih/1986cnr/stenprot/009schuz/s009011.htm
Kdy: 1988-04-12
Co: ská vláda v této době zakládá také modernizaci a rozšíření výroby gumových operačních rukavic a dalších ochranných výrobků v souvislosti s prudce narůstající potřebou v souvislosti s výskytem choroby AIDS. I když nemáme devizové prostředky navíc v české oblastí, řešíme tyto otázky přednostně a v užijeme k tomu i podnikatelské formy ve světě obvyklé, jako jsou devizově návratné úvěry, leasing, pronájem
***
Kdo: Poslanec Jiří Fleyberk
Kde: 1986cnr_stenprot_013schuz_s013009.htm
URL: https://www.psp.cz/eknih/1986cnr/stenprot/013schuz/s013009.htm
Kdy: 1988-12-20
Co: ní zabezpečení téměř 4,8 mld Kčs, na školství 16,3 mld Kčs, na zdravotnictví 21,4 mld - z toho je téměř 1/3 určena na krytí výdajů za léky a speciální zdravotnický materiál a 400 mil. Kčs na prevenci AIDS, 3,1 mld Kčs na oblast kultury, včetně nezbytné údržby a opravy h

In [63]:
hledej(r"xenofob")

Kdo: Prezident České republiky Václav Havel
Kde: 1993ps_stenprot_006schuz_s006002.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/006schuz/s006002.htm
Kdy: 1993-02-23
Co:  své vlastní frustrace, aby si pak své otřesené sebevědomí léčili jeho pronásledováním. Velký vzrůst kriminality a organizovaného zločinu, kolektivní násilí, nesnášenlivost, rasismus, antisemitismus, xenofobie a lhostejnost k nim, šířící se korupce, zlatokopecká horečka a názor, že život je džungle a člověk člověku musí být vlkem - to jsou zřejmě jen nejznámější projevy onoho zvláštního stavu, do něhož s
***
Kdo: Místopředseda PSP Jiří Vlach
Kde: 1993ps_stenprot_008schuz_s008001.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/008schuz/s008001.htm
Kdy: 1993-04-20
Co: izraelského parlamentu se obrátil dopisem na Poslaneckou sněmovnu se žádostí, aby vzpomněla židovského povstání ve Varšavském ghettu, které vypuklo 19. dubna 1943 a vyjádřila tímto svůj protest proti xenofobii, rasismu a antisemitismu. Dovolte krátké p

In [64]:
hledej(r"domác\w{0,10} násilí")

Kdo: Poslankyně Jana Volfová
Kde: 1998ps_stenprot_029schuz_s029083.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/029schuz/s029083.htm
Kdy: 2000-11-30
Co: lím, a proto i já doporučuji k projednání do prvního(?) čtení. Chtěla bych ale obrátit vaši pozornost k problému, který se bytostně dotýká nebo je velmi spojován s problémem znásilnění. To je problém domácího násilí obecně. V ČR tak jako ve světě se odhaduje podíl 
***
Kdo: Poslanec Vilém Holáň
Kde: 1998ps_stenprot_046schuz_s046320.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/046schuz/s046320.htm
Kdy: 2002-02-08
Co: Vážený pane místopředsedo, vážené kolegyně, vážení kolegové, náš návrh se zabývá problematikou domácího násilí. Domácí násilí je už dlouhou dobu vážným společenským problémem, který však zůstává do značné míry stranou legislativní pozornosti. Na základě současné úpravy trestního zákona lze stíhat pouze člověk
***
Kdo: Poslanec Vilém Holáň
Kde: 1998ps_stenprot_046schuz_s046321.htm
URL: https://www.psp.cz/eknih/1998

In [65]:
hledej(r"oligarch")

Kdo: Posl. Pospíšil
Kde: 1918ns_ps_stenprot_019schuz_s019006.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/019schuz/s019006.htm
Kdy: 1919-01-22
Co: ta říše nedovedla chápati, že lidé 20. století nedají se hnáti jako stádo na jatky a nedají si diktovat ani jedincem, ani snad několika lidmi, že tedy v dnešních dobách nemá místa ani autokracie, ani oligarchie, ale že nemá v dnešních dobách místa ani ochlokracie, ani luzovláda, která po případě za určitých okolností může mnohé státy také rozvrátiti, a že místo tu má jedině a pouze demokracie, a to demokr
***
Kdo: Zpravodaj člen Národního shromáždění dr. Hajn
Kde: 1918ns_ps_stenprot_040schuz_s040001.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/040schuz/s040001.htm
Kdy: 1919-03-27
Co: sou jako spojené nádoby. Ve všech stejně zároveň, ať autorita anebo bezvýznamnost jest na stejné výši, ač jsme-li stále v normálních, konstitučních poměrech a nikoliv snad v utajeném absolutismu nebo oligarchii nebo v diktatuře. Národní shromáždě

In [66]:
hledej(r"Babiš")

Kdo: Místopředsedkyně PSP Miroslava Němcová
Kde: 2002ps_stenprot_047schuz_s047029.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/047schuz/s047029.htm
Kdy: 2005-09-20
Co:  tuto roli pak sehrál Agrofert. Kluzek říká, že je možné, že obě firmy prošly jakýmsi konkursem na to, kdo se touto spojkou stane. K zisku jednotlivých stran transakce Kluzek uvádí, že se domnívá, že Babiš na ní určitě nemohl finančně ztratit, role společnosti Seta se mu zdá podivná. K rozsahu korupce uvádí - ze záznamu vyšetřovací komise cituji: Již podle dřívějšího svědectví došlo na obou stranách tr
***
Kdo: Poslanec Jozef Kochan
Kde: 2002ps_stenprot_047schuz_s047112.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/047schuz/s047112.htm
Kdy: 2005-09-22
Co: dvě, které s kauzou souvisí, neboť jí předcházejí. Vážený pane předsedo vlády Paroubku, mohl byste sdělit Poslanecké sněmovny, proč při první privatizaci Unipetrolu, kterou vyhrála firma pana Andreje Babiše a který později od této smlouvy ustoupil, nebyla mu Fo

In [67]:
hledej(r"(?i)dezinf")

Kdo: Prezident Gottwald pri tejto príležitosti povedal
Kde: 1948ns_stenprot_040schuz_s040003.htm
URL: https://www.psp.cz/eknih/1948ns/stenprot/040schuz/s040003.htm
Kdy: 1950-02-22
Co: italistických štátoch, si nijako vojnu nepraje. A to je príčina, prečo podnecovatelia novej vojny vrhajú sa naň s obrovským aparátom. "V Spojených štátoch amerických vládne prekvapujúci stav verejnej dezinformovanosti", povedal raz prezident Roosevelt. A s týmto stavom zámerne udržovanej 
***
Kdo: Předsedkyně Sněmovny lidu MUDr. S. Pennigerová
Kde: 1969fs_sl_stenprot_004schuz_s004002.htm
URL: https://www.psp.cz/eknih/1969fs/sl/stenprot/004schuz/s004002.htm
Kdy: 1969-10-15
Co: í mnozí pocit, že byli oklamáni. Avšak nevidí ještě dosud skutečné viníky, a to ty, kteří s nimi kalkulovali a v období vypjatých politických situací je měli připravené jako želízka v ohni. Politická dezinformace je u mládeže zvlášť silná. Je u ní i silná deziluze. Chybí jí dnes vzory, ideály, někdo nebo něco, za čím by mohla jít a č

In [68]:
hledej(r"(?i)fake news")

Kdo: Ministr zdravotnictví ČR Miloslav Ludvík
Kde: 2013ps_stenprot_058schuz_s058020.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/058schuz/s058020.htm
Kdy: 2017-05-10
Co: e překvapovat, jakým způsobem se v téhle zemi vytvářejí nebo vyrábějí kauzy. Z tohoto se udělá kauza. Finále je co? Nic. Kdybych byl blonďák a seděl jsem o něco západněji, tak bych to nazval typickou fake news. Ale já touto cestou cítím, a omlouvám se, že tady zdržuji, cítím trochu povinnost se za prvé obhájit a za druhé říci, že tenhle způsob obchodování s informacemi je strašný, a skoro je mi líto, že to
***
Kdo: Poslankyně Helena Langšádlová
Kde: 2013ps_stenprot_057schuz_s057371.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/057schuz/s057371.htm
Kdy: 2017-06-08
Co:  Koudelka na serveru HlídacíPes. Z rozhovoru vyplývají mnohé negativní skutečnosti. Jedná se o nárůst webových stránek, na kterých se velmi často setkáváme s proruskou propagandou, ale i s tím, že se fake news vyskytují i ve zdánlivě seriózních mé

In [69]:
hledej(r"(?i)dezinform")

Kdo: Prezident Gottwald pri tejto príležitosti povedal
Kde: 1948ns_stenprot_040schuz_s040003.htm
URL: https://www.psp.cz/eknih/1948ns/stenprot/040schuz/s040003.htm
Kdy: 1950-02-22
Co: italistických štátoch, si nijako vojnu nepraje. A to je príčina, prečo podnecovatelia novej vojny vrhajú sa naň s obrovským aparátom. "V Spojených štátoch amerických vládne prekvapujúci stav verejnej dezinformovanosti", povedal raz prezident Roosevelt. A s týmto stavom zámerne udržovanej 
***
Kdo: Předsedkyně Sněmovny lidu MUDr. S. Pennigerová
Kde: 1969fs_sl_stenprot_004schuz_s004002.htm
URL: https://www.psp.cz/eknih/1969fs/sl/stenprot/004schuz/s004002.htm
Kdy: 1969-10-15
Co: í mnozí pocit, že byli oklamáni. Avšak nevidí ještě dosud skutečné viníky, a to ty, kteří s nimi kalkulovali a v období vypjatých politických situací je měli připravené jako želízka v ohni. Politická dezinformace je u mládeže zvlášť silná. Je u ní i silná deziluze. Chybí jí dnes vzory, ideály, někdo nebo něco, za čím by mohla jít a č

In [70]:
hledej(r"(?i)elektron\w{0,10} pošt")

Kdo: Poslanec Jiří Payne
Kde: 1996ps_stenprot_012schuz_s012077.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/012schuz/s012077.htm
Kdy: 1997-07-10
Co: ídit z jednoho místa, výrazně větší. Zatímco ve starých dobách rakousko- uherské byrokracie bylo možné z jednoho místa řídit maximálně 10 -12 jednotek, v dnešní době při existenci telefonů, telefaxů, elektronické pošty, počítačových sítí - buďme si vědomi toho, že každý okres je připojen počítačovou sítí na Ministerstvo vnitra - ten počet roste z deseti až dvanácti před sto lety na nějakých osmdesát až sto. Zkusme
***
Kdo: Poslanec Oldřich Kužílek
Kde: 1996ps_stenprot_023schuz_s023063.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/023schuz/s023063.htm
Kdy: 1998-03-25
Co: je přirozené, že tento návrh - a já to přičítám určité novosti problematiky - vyvolal řadu otázek. Proto jsem si dovolil nejčastější z nich sepsat, poskytnout k nim určitý výklad a vám všem je zaslat elektronickou poštou. Když si otevřete počítač, najdete zprávu, k

In [71]:
hledej(r"(?i)e-mail")

Kdo: Poslanec Jaromír Schling
Kde: 1998ps_stenprot_015schuz_s015175.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/015schuz/s015175.htm
Kdy: 1999-07-02
Co:  přednést. Jedná se o pozměňovací návrh k § 5 odstavec 1 písmeno i), které by podle přijatého pozměňovacího návrhu mělo znít: Název, sídlo, telefon, popř. dálnopis, fax nebo adresu telefonické pošty (e-mail) zadavatele a jméno styčného pracovníka. Druhý návrh je k § 50 odstavec 1 písmeno f), kde hospodářský výbor schválil pozměňovací návrh - písmeno f) vypustit. Děkuji.
***
Kdo: Poslankyně Kateřina Dostálová
Kde: 1998ps_stenprot_019schuz_s019012.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/019schuz/s019012.htm
Kdy: 1999-11-30
Co: ávání v oboru restaurování. A dále bych ráda řekla ještě jednu poznámku - byli jsme všichni, domnívám se, že i vy, kolegové, seznámeni s prohlášením Asociace restaurátorů - alespoň já jsem to dostala e-mailem -, a přijde mi docela pikantní, když jsem si přečetla, že pod tímto prohlášením Asociace re

In [72]:
hledej(r"(?i)instagra")

Kdo: Senátor Zdeněk Nytra
Kde: 2017ps_stenprot_026schuz_s026089.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/026schuz/s026089.htm
Kdy: 2019-01-23
Co:  citováno z dopisu, že umělci nedostali nic zaplaceno, tak se ptám, jak je to možné, když každý provozovatel nebo držitel licence rádiového nebo televizního vysílání musí mít uzavřenou smlouvu s OSA, Instagramem a případně dalším, takže ti umělci museli dostat zaplaceno, nebo musí dostat zaplaceno. A nechci se tady dostávat do polemiky, jestli ty peníze zůstávají v OSA, nebo nezůstávají v OSA. To opravdu 
***
Kdo: Poslanec Dominik Feri
Kde: 2017ps_stenprot_062schuz_s062098.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/062schuz/s062098.htm
Kdy: 2020-10-22
Co: kové konferenci vždycky řeknete "my to zlepšíme", nový ministr řekne "my to zlepšíme", ale je to pořád stabilní, ale stabilně špatné. Ze samé zoufalosti jsem se rozhodl ta opatření komunikovat sám na Instagramu. Už to sleduje 700 tisíc lidí. Ale to samo nestačí a stačit nemůž

In [73]:
hledej(r"(?i)facebook")

Kdo: Poslanec Ondřej Liška
Kde: 2006ps_stenprot_059schuz_s059199.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/059schuz/s059199.htm
Kdy: 2009-06-17
Co: otvrzeno či vyvráceno, zda maturita je, nebo není dobře připravena. To znamená aby byl důsledně hájen zájem těch, kteří budou první generací, která bude skládat státní maturitní zkoušku. To, že se na facebooku přihlásilo několik desítek tisíc mladých lidí, kteří mají strach z něčeho nového, o čem přesně všechno nemohou vědět, protože je to velmi složitý proces, do kterého vidí skutečně jen několik desítek
***
Kdo: Poslankyně Olga Zubová
Kde: 2006ps_stenprot_060schuz_s060071.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/060schuz/s060071.htm
Kdy: 2009-09-09
Co: louchat jim. Státní maturitu odmítá stále víc lidí. Svůj nesouhlas vyjádřilo přes 20 tisíc lidí v petici za odklad státních maturit. Bezmála 60 tisíc členů obsahuje každá ze dvou skupin sociální sítě Facebook, které vyjadřují svůj nesouhlas s projektem, který jste vy, posla

In [74]:
hledej(r"(?i)internet")

Kdo: Poslankyně Květoslava Čelišová
Kde: 1993ps_stenprot_037schuz_s037019.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/037schuz/s037019.htm
Kdy: 1995-12-07
Co: čnou překážkou zůstane stavební povolení. Opravdu chceme, aby o tak strategicky důležité věci, jakou zcela jistě kabelové rozvody jsou, a nejde pouze o televizní vysílání, ale i o informační dálnici, Internet apod., rozhodoval jeden úředník stavebního odboru obecního úřadu? Jinou věcí je, že v kabelových rozvodech mohou vysílat, resp. vytvářet programy i ti, kteří sami vlastníky rozvodu nejsou. O liberali
***
Kdo: Ministr kultury ČR Jaromír Talíř
Kde: 1996ps_stenprot_010schuz_s010026.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/010schuz/s010026.htm
Kdy: 1997-03-27
Co: najdeme řešení pro jednotlivá zde definovaná témata a pro definované problémové okruhy. Myslím si, že ještě bude nutné, a zapomněl jsem tu uvést, že tato legislativa se také dotkne datových sítí typu Internet a že tady bude třeba také definovat určité ter

In [75]:
hledej(r"(?i)kde udělali soudr")

Kdo: Poslanec David Rath
Kde: 2010ps_stenprot_009schuz_s009076.htm
URL: https://www.psp.cz/eknih/2010ps/stenprot/009schuz/s009076.htm
Kdy: 2010-12-09
Co: a, Dánska, k nám neutíkají manažeři a podnikatelé s vysokými příjmy, aby tam nemuseli platit vysoké daně. Takže dámy a pánové, vám ten pravicový model v praxi nějak moc v České republice nefunguje. Někde udělali soudruzi z ODS asi chybu.
***
Kdo: Poslanec Radim Fiala
Kde: 2021ps_stenprot_010schuz_s010003.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/010schuz/s010003.htm
Kdy: 2022-02-17
Co:  nyní pohybuje těsně pod hranicí 100 eur za tunu CO2, přičemž dle prognóz Evropské unie se měla tato cena v roce 2025 pohybovat okolo 26 eur a v roce 2030 pak okolo 30 eur za tunu CO2. A tak se ptám, kde udělali soudruzi v Bruselu chybu? Mimochodem od začátku loňského roku se jedná o zvýšení o více než 200 %. A oproti roku 2020 jde už o čtyřnásobný nárůst. K těmto vysokým cenám povolenek ovšem přispívá nejen zdraž
***
Kdo: Poslanec Radim Fiala
K

In [76]:
hledej(r"(?i)švejk")

Kdo: Posl. Tausik (pokračuje)
Kde: 1920ns_ps_stenprot_306schuz_s306009.htm
URL: https://www.psp.cz/eknih/1920ns/ps/stenprot/306schuz/s306009.htm
Kdy: 1924-11-26
Co: é funkci v Praze jednal tak, jako jednal v Košicích, tak mohou nastati poměry, ze kterých mnoha pánům bude boleti hlava. Chci napřed krátce jenom naznačením uvésti několik extravagancí tohoto vojáka "Švejka", jichž svědky jsme byli v poměrně krátké době jeho účinkování jako velitele 11. pěší divise v Košicích. Z činnosti Gajdovy v Košicích opakuji jen tyto okolnosti a skutečnosti: Důvěrné styky s Klímo
***
Kdo: Posl. dr Chvalkovský
Kde: 1925ns_ps_stenprot_045schuz_s045007.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/045schuz/s045007.htm
Kdy: 1926-10-20
Co: stopadu 1924 v rozpočtovém výboru: "Nemám jiného přání, než abych i s oposicí shledal se u kladné práce." (Výkřiky komunistických poslanců.) Já se k vám také vrátím. (Výkřiky komunistických poslanců: Švejk, II. díl!) 
***
Kdo: Místopředseda inž. Dostálek (zvoní)
K

In [77]:
hledej(r"(?i)uhlík\w{0,10} neutr")

Kdo: Ministr životního prostředí ČR Richard Brabec
Kde: 2017ps_stenprot_032schuz_s032034.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/032schuz/s032034.htm
Kdy: 2019-06-26
Co: Tady se to smrsklo, odpusťte mi to slovo, jenom na debatu, zda dosáhneme v roce 2050 takzvané uhlíkové neutrality, nebo nedosáhneme, nebo kdy toho dosáhneme. Když se dneska sta procent lidí zeptáte, ale myslím, že i lidí, kteří na to mají třeba i nějaké odbornější vzdělání, co to vlastně ta 
***
Kdo: Předseda vlády ČR Andrej Babiš
Kde: 2017ps_stenprot_039schuz_s039114.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/039schuz/s039114.htm
Kdy: 2019-11-28
Co: lné elektrárny v EU. To je nepřijatelné. Absolutně nepřijatelné! A my máme jasnou strategii. My jsme odpovědní. My říkáme, že v 2030 budeme na 43 %, že v 2050 budeme na 80 %, a na to, abychom dosáhli uhlíkovou neutralitu, potřebujeme vydat možná bilion. Možná bilion. A my hlavně potřebujeme jádro. My jsme zaspali dobu. Já jsem publikoval na mém Facebooku i

In [78]:
hledej(r"(?i)skleníko\w{0,10} (plyn|efe)")

Kdo: Ministr životního prostředí ČR Ivan Dejmal
Kde: 1990cnr_stenprot_024schuz_s024002.htm
URL: https://www.psp.cz/eknih/1990cnr/stenprot/024schuz/s024002.htm
Kdy: 1991-11-06
Co: jí příchod a udělali opatření, kterými jí budeme čelit. Chtěl bych zde zmínit jeden z programů našeho ministerstva, který jde vstříc výše ohlášenému požadavku. Změna klimat v důsledku vypouštění tzv. skleníkových plynplyn
***
Kdo: Ministr průmyslu a obchodu ČR Vladimír Dlouhý
Kde: 1993ps_stenprot_031schuz_s031004.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/031schuz/s031004.htm
Kdy: 1995-05-23
Co: lektráren zatěžuje životní prostředí zcela minimálně, když je zcela vyloučena produkce oxidu dusíku i síry. Stejně tak je vyloučena i produkce kysličníku uhličitého, který se výrazně podílí na tvorbě skleníkového efeefe
***
Kdo: Poslanec Tomáš Páv
Kde: 1996ps_stenprot_005schuz_s005035.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/005schuz/s005035.htm
Kdy: 1996-10-04
Co:  životního prostředí. Konečně dnes již

In [79]:
hledej(r"(?i)globál\w{0,10} oteplo")

Kdo: Poslanec Tomáš Páv
Kde: 1996ps_stenprot_005schuz_s005035.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/005schuz/s005035.htm
Kdy: 1996-10-04
Co: itelné zdroje energie mohou poskytnout pouze lokálně podmíněné doplňkové řešení. Hlavní zátěž bude, ať se nám to líbí nebo ne, ve spalování fosilních paliv a v jaderné energetice. S ohledem na efekty globálního oteplování se svět bude muset obracet zpátky k jaderné energetice. Hovořil jsem o jaderné energetice jako o nejbezpečnější technologii získávání významného množství energie. To platí ovšem pouze tam, kde vy
***
Kdo: Poslanec Zdeněk Švrček
Kde: 1998ps_stenprot_008schuz_s008019.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/008schuz/s008019.htm
Kdy: 1999-01-12
Co:  neodpovídají při nezpevněných korytech snad ani desetileté vodě. Upozorňuji, že při současných anomáliích počasí, které podle dlouhodobých sledování periodicky navštěvují Zemi a nyní jsou zesíleny i globálním oteplováním, které loňská mezinárodní meteorologická konfe

In [80]:
hledej(r"(?i)solár\w{0,10} energ")

Kdo: Společný zpravodaj výborů Sněmovny lidu A. Hůla
Kde: 1981fs_sl_stenprot_002schuz_s002004.htm
URL: https://www.psp.cz/eknih/1981fs/sl/stenprot/002schuz/s002004.htm
Kdy: 1982-10-20
Co: sobní a věcné podmínky a předpoklady. Výbor pro zemědělství a výživu projednával racionalizaci dopravy a její využívání v zemědělství a zabýval se zkušenostmi z netradičního využívání energie jako je solární energie, odpadní teplo z kompresních stanic tranzitního plynovodu a možnostmi výroby bioplynu. Naše zemědělství se na celkové spotřebě paliv a energie v ČSSR podílí 5,5 %. a potravinářský průmysl 5,2 %. Na
***
Kdo: Poslanec G. Mede
Kde: 1981fs_sn_stenprot_004schuz_s004005.htm
URL: https://www.psp.cz/eknih/1981fs/sn/stenprot/004schuz/s004005.htm
Kdy: 1984-10-04
Co: cnosťou práve na úseku dokončovacích prác. Na tomto úseku chýba cieľavedomá a efektívna spolupráca medzi stavebníctvom a strojárskymi odvetviami. Nevyužíva sa sálavé vykurovanie, tepelné čerpadlá ani solárna energia - i keď na tomto úsek

In [81]:
df.filter(pl.col('mluvci').str.contains('Gottwald') & pl.col('text').str.contains('krk'))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,u32
"""Posl. Gottwald (pokračuje)""","""1929ns_ps_stenprot_007schuz_s007003.htm""",null,""". . . kterého bude dosaženo teprve tehdy, až budete smeteni. (Posl. Zeminová: My jsme si stát budovali a ne vy! - Posl. Slavíček: Vy máte o to malou z…",21,"""ps""",1929,7,3,1929-12-21,"""NS RČS 1929-1935, Poslanecká sněmovna""","""7. schůze""",true,true,true,183
"""Posl. Gottwald""","""1929ns_ps_stenprot_174schuz_s174001.htm""",null,"""V poslední schůzi před týdnem jsme navrhli, aby o prohlášení ministra Dostálka byla zahájena okamžitě debata. Vládní koalice, socialistické milionáře …",13,"""ps""",1929,174,1,1932-04-07,"""NS RČS 1929-1935, Poslanecká sněmovna""","""174. schůze""",true,true,true,886
"""Posl. Gottwald (pokračuje)""","""1929ns_ps_stenprot_213schuz_s213012.htm""",null,"""... spolu s ostatní plejádou podobných socialistů, pak už toto osobní složení nové vlády ukazuje na její kurs. Bude to jiný kurs než kurs staré vlády?…",4,"""ps""",1929,213,12,1932-11-04,"""NS RČS 1929-1935, Poslanecká sněmovna""","""213. schůze""",true,true,true,2688
"""Posl. Gottwald""","""1935ns_ps_stenprot_068schuz_s068009.htm""",null,"""U příležitosti generální debaty o státním rozpočtu považujeme za nutné obrátit pozornost k některým zahraničně-politickým otázkám, které se dotýkají z…",2,"""ps""",1935,68,9,1936-12-01,"""NS RČS 1935-1938, Poslanecká sněmovna""","""68. schůze""",true,true,true,3256


In [82]:
hledej(r"zakroutit krkem")

Kdo: Poslanec Vladimír Struska
Kde: 1986cnr_stenprot_026schuz_s026003.htm
URL: https://www.psp.cz/eknih/1986cnr/stenprot/026schuz/s026003.htm
Kdy: 1990-04-25
Co: rtel nad tím, že již od roku 1929 vedení KSČ zrazovalo zájmy svého lidu a dopouštělo se velezrady, jak nám dosvědčuje Klement Gottwald, když prohlásil ve sněmovně: "Ano, my se učíme v Moskvě, jak vám zakroutit krkem." Soud dějin zjistí, od kdy začali připravovat státní převrat. Já sám jsem v roce 1947 byl svými přáteli z KSČ vyzván slovy: "Dej se k nám. Máme zbraně a budeme dělat převrat." Soud dějin odsoudí ved
***
Kdo: Předseda vlády ČR Miloš Zeman
Kde: 1998ps_stenprot_025schuz_s025087.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/025schuz/s025087.htm
Kdy: 2000-05-18
Co: mperialismu. Závěrem mi dovolte, abych jako poslední větu své odpovědi lehce parafrázoval citát vašeho oblíbeného klasika Klementa Gottwalda: Jezdíme do Washingtonu, abychom se naučili, jak tunelářům zakroutit krkem. Děkuji za pozornost.
***
Kdo: Poslane

In [83]:
hledej(r"DNA")

Kdo: Poslanec Libor Ambrozek
Kde: 1998ps_stenprot_021schuz_s021179.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/021schuz/s021179.htm
Kdy: 2000-01-25
Co:  jakéhokoli nakládání s geneticky modifikovanými organismy. Genové manipulace jsou postupy legitimní, které se v laboratorní praxi využívají od poloviny 70. let. Jejich éra začala s objevem struktury DNA, která je odpovědná za přenos genetické informace v buňce. Nejprve se používaly k čistě teoretickým výzkumům, jak už to v této oblasti bývá, později se objevily obrovské možnosti nejprve v biomedicín
***
Kdo: Poslanec Rostislav Čevela
Kde: 1998ps_stenprot_024schuz_s024089.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/024schuz/s024089.htm
Kdy: 2000-04-06
Co: ogeneze, byl by Merežkovsky zapomenut. Margulisová totiž prokázala, že eukaryotická buňka je v podstatě mnohobuněčný tvor a buněčné organely, jako např. mitochondrie či choroplasty - nositelé vlastní DNA - byly kdysi samostatné organismy, tzv. prokaryonta. Vetřelci a hostit

In [84]:
hledej(r"DVD")

Kdo: Poslanec Zdeněk Švrček
Kde: 1998ps_stenprot_017schuz_s017060.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/017schuz/s017060.htm
Kdy: 1999-10-13
Co: žnost upravit si vše tak, aby z ložné plochy jakéhokoliv auta mohl kdokoli prodávat cokoli, avšak rád bych svým dalším návrhem ponechal právo pojízdným prodejnám firem, jako jsou Ledík, Family Frost, DVD nebo MAN, prodávat dosavadním způsobem a nevystavoval je možnému nátlaku použitím dosud navržené úpravy.
***
Kdo: Poslanec Aleš Rozehnal
Kde: 1998ps_stenprot_022schuz_s022018.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/022schuz/s022018.htm
Kdy: 2000-02-22
Co: ýlučného distribučního práva je odůvodnitelné v rámci jednotného trhu, nikoliv však ve vztahu k třetím státům. Na tomto principu je založen i celosvětový systém distribuce a ochrany digitálních disků DVD, jejichž volná distribuce je technickými opatřeními omezena pouze na okruh stanoveného regionu. Teritoriální princip se uplatňuje též v Evropské unii, kde je zachováno a

In [85]:
hledej(r"(?i)delulu")

In [86]:
hledej(r"(?i)[^\w]Gaz[aey][^\w]")

Kdo: Poslanec SL J. Lichner
Kde: 1976fs_slsn_stenprot_013schuz_s013012.htm
URL: https://www.psp.cz/eknih/1976fs/slsn/stenprot/013schuz/s013012.htm
Kdy: 1979-10-24
Co: ská zmluva uvoľnila izraelskej vláde ruky. Dokazuje to jednak izraelská politika na okupovaných územiach, postoj na rokovaniach o autonómii pre arabské obyvateľstvo na západnom brehu Jordánu a v pásme Gazy. Zvlášť provokatívna a nebezpečná je izraelská agresia proti susednému Libanonu, ktorej súčasťou je pokus o genocídu palestínskeho ľudu, toľko skúšaného v posledných desaťročiach. Z doterajšieho prieb
***
Kdo: Poslanec Josef Krejsa
Kde: 1996ps_stenprot_017schuz_s017041.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/017schuz/s017041.htm
Kdy: 1997-12-10
Co: dloužen do 31. ledna 1998. Konec citátu. Tedy výzva bez sankcí, vše zůstalo nevyslyšeno. Další rezoluce Rady bezpečnosti je nejčerstvějšího data a týká se zákazu výstavby bytů na okupovaných oblastech Gazy. Tuto rezoluci vydala Rada bezpečnosti opět na rozdíl od rez

In [87]:
hledej(r"(?i)irozhlas")

Kdo: Předseda vlády ČR Andrej Babiš
Kde: 2017ps_stenprot_007schuz_s007101.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/007schuz/s007101.htm
Kdy: 2018-03-01
Co:  Ne, tady je: Murína kvůli kauze úniku informací Bereta vyslýchali policisté. Důstojník uvedl, že se stýkal s Gáboríkem, jehož policie stíhá kvůli obchodování s utajovanými informacemi. Četl jste to? iRozhlas! Antibabišovský server! Každý den kopou do Babiše. (Hovoří pobaveně.) No tak to je strašný, ne, co řekli. Tak si to nastudujte, pane kolego, a neříkejte, že je to Čapí hnízdo. A nikoho nebudeme dopor
***
Kdo: Poslanec Dominik Feri
Kde: 2017ps_stenprot_016schuz_s016098.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/016schuz/s016098.htm
Kdy: 2018-06-28
Co: alá není, tak nemůže osvětlit i záležitosti kolem výkonu koncipientury 300 km od svého bydliště, nebo třeba osvětlit a uvést na pravou míru informace, které vyšly před půl hodinou v článku na serveru iROZHLAS s titulkem Nastupující šéfka justice Taťána Malá podle ex

In [88]:
hledej(r"(?i)masturb")

Kdo: Poslanec Karel Vymětal
Kde: 1998ps_stenprot_046schuz_s046420.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/046schuz/s046420.htm
Kdy: 2002-02-14
Co: Děkuji za slovo. Ano, chci předložit procedurální návrh. Dámy a pánové, nechci mluvit o mikrofonové masturbaci, i když by to někdo mohl tak cítit, já to tak necítím. Já vnímám, že do současné chvíle byla přednesena řada velmi rozsáhlých a hlubokých projevů, jako například pana poslance Sloty, promiňte Neča
***
Kdo: Poslanec Petr Kudela
Kde: 2013ps_stenprot_027schuz_s027122.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/027schuz/s027122.htm
Kdy: 2015-04-30
Co: přes užívání sladkostí k čištění zubů, celkovou hygienu, škodlivost kouření a nebezpečí drog až - cituji - jací se mi líbí kluci, jaké se mi líbí holky, menstruace, měsíčky, menzes, čmýra, onanování, masturbace, prstění. Prosím, dětem v 5. třídě! Informovat školáky o stavbě lidského těla a o jeho fyziologii a reprodukci je zcela namístě, ale je potřeba to dělat kultivovanou

In [89]:
hledej(r"(?i)Donald\w{0,3} Trump")

Kdo: Místopředseda vlády a ministr vnitra ČR Radek John
Kde: 2010ps_stenprot_013schuz_s013064.htm
URL: https://www.psp.cz/eknih/2010ps/stenprot/013schuz/s013064.htm
Kdy: 2011-02-03
Co:  pan Jahodář měl ovlivnit nějaký tendr. Podle mých informací se žádný tendr na přepravu hotovosti a cenin v České poště nepřipravuje. Takže to je úplně stejné, jako kdybyste říkal, že pan Jahodář pro Donalda Trumpa chce ovlivnit nějakou zakázku, jenomže ona není. Tak co budeme dělat, pane doktore? Všechny ty vaše řeči o mém soukromém podnikání - a ještě mě přistrkujete do MEDEA KULTUR, kde jsem nikdy nebyl, ne
***
Kdo: Místopředseda vlády ČR a ministr financí Andrej Babiš
Kde: 2013ps_stenprot_043schuz_s043004.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/043schuz/s043004.htm
Kdy: 2016-03-23
Co: , aby realizovali smysl svého života a předali své dílo svým dětem, je zamezeno se podílet na veřejném životě ve prospěch všech občanů. Přitom všude na světě je trend opačný. V Americe máme ve finále Donalda T

In [90]:
hledej(r"(?i)proletář",sestupne=True)

Kdo: Poslankyně Alena Schillerová
Kde: 2021ps_stenprot_115schuz_s115025.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/115schuz/s115025.htm
Kdy: 2024-10-22
Co: chterová - teď tady není, takže nemusím říkat prostřednictvím - ještě pořád je místopředsedkyní Sněmovny? S ministerským platem, s autem s řidičem, s aparátem? Pořád tam - jak je to možné? Vy, takoví proletáři! (Pobavení v sále.) Tak já očekávám rezignaci v co nejkratší době! Děkuju. (Poslanci volají, potlesk z lavic hnutí ANO.)
***
Kdo: Poslankyně Eva Dundáčková
Kde: 2002ps_stenprot_054schuz_s054139.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/054schuz/s054139.htm
Kdy: 2006-03-10
Co: nost v lidově demokratickém Československu - Rudé právo 1956, autor Zdeněk Jičínský: Buržoazní právo a s ním i licoměrná zvetšelá buržoazní ideologie o nadtřídním právu je odklizena na smetiště dějin proletářskou revolucí, v níž se vytváří nové, revoluční 
***
Kdo: Poslanec Josef Vondruška
Kde: 2002ps_stenprot_051schuz_s051607.htm
URL: http

In [91]:
hledej(r"(?i)skinhead",sestupne=True)

Kdo: Místopředseda PSP Ivan Langer
Kde: 2002ps_stenprot_047schuz_s047011.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/047schuz/s047011.htm
Kdy: 2005-09-20
Co: tázku, v čem se lišil právní stav věci, když se tak zásadně lišil faktický stav policejního zákroku. Otázka osmá: Jaký je důvod pro odlišný přístup policie v případech shromáždění neonacistů, průvodu skinheadů a anarchistů ulicemi, při shromáždění komunistů na Letné, při akci CzechTek, při demonstraci u budovy Ministerstva vnitra a posprejování její fasády?
***
Kdo: Místopředseda PSP Ivan Langer
Kde: 2002ps_stenprot_046schuz_s046007.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/046schuz/s046007.htm
Kdy: 2005-08-16
Co:  nás všechny musí zajímat odpověď na otázku, zda policie měří v této zemi všem podle jednoho metru, tedy proč v případě CzechTeku postupovala takto, proč postupuje odlišným způsobem v případě průvodu skinheadů a anarchistů ulicemi našich měst, proč zareagovala na shromáždění Komunistické strany Čech a Morav

In [92]:
hledej(r"Paroub",sestupne=True)

Kdo: Poslanec Andrej Babiš
Kde: 2021ps_stenprot_146schuz_s146011.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/146schuz/s146011.htm
Kdy: 2025-06-25
Co: nou, zákulisní osobnost brněnské ODS i potenciální hrozbu. Topolánek i Nečas měli z Blažka obavy. Kdo by neměl, takový kmotr! V letech 2008 až 2013 dominovali ODS a celostátní politice tři muži: Jiří Paroubek, Petr Nečas a Mirek Topolánek. Oba zmínění premiéři za ODS Blažkovi nikdy nevěřili, museli ale respektovat jeho vliv a schopnost budovat nemožné koalice ve složitém prostředí, často protichůdných z
***
Kdo: Ministr financí ČR Zbyněk Stanjura
Kde: 2021ps_stenprot_136schuz_s136063.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/136schuz/s136063.htm
Kdy: 2025-04-16
Co: a neprosadila opozice. To v době, mám to od pamětníků stojedničky, mám pocit, že vláda, to bylo to období, kdy měli sociální demokraté tři premiéry, to znamená Vladimír Špidla, Stanislav Gross a Jiří Paroubek, jich bylo přesně 101, tak aby se shromáždili v konkrét

In [93]:
hledej(r"(?i)parní\w{0,5} stroj",sestupne=True)

Kdo: Poslanec Václav Klaus
Kde: 2017ps_stenprot_051schuz_s051024.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/051schuz/s051024.htm
Kdy: 2020-06-18
Co: tě, všechno možné se vezlo, ale že by tatínek jezdil do práce automobilem, bylo vyloučené, protože ten benzin byl daleko dražší. Já ještě řeknu jednu věc. Když jsme u těch technologií, kupodivu třeba parní stroje byly docela efektivní. Když jsem mluvil o té chalupě, tak do Benešova jezdila elektrická lokomotiva, ale pak už elektrika nebyla, nevím, jestli je dneska teda, asi jo, a pak se jezdilo parní lokomot
***
Kdo: Ministryně pro místní rozvoj ČR Klára Dostálová
Kde: 2017ps_stenprot_016schuz_s016092.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/016schuz/s016092.htm
Kdy: 2018-06-28
Co: kraticky dané nebo zbytečně administrativně nákladné nebo jsou už přežité. Už jsem to i jednou říkala na výboru pro veřejnou správu, že my se například zabýváme zákonem z roku "devatenáct set něco" o parních strojích. Samozřejmě dneska 
***
Kdo: 

In [94]:
hledej(r"(?i)telegra[mf]",sestupne=True)

Kdo: Poslanec Vladimír Zlínský
Kde: 2021ps_stenprot_144schuz_s144098.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/144schuz/s144098.htm
Kdy: 2025-06-18
Co: rmací, byl vynález knihtisku, který pak vedl k tomu, že se výrazně zvýšila vzdělanost populace, schopnost číst a psát, a to samozřejmě podnítilo vědecký a technický rozvoj, který dále vedl k vynálezu telegrafu, rádiového a televizního vysílání, které umožňovaly rychlé, většinou tedy jednosměrné šíření typu informací směrem k jejich uživatelům a vedly k posílení jak demokratických, tak propagandistických p
***
Kdo: Poslanec Vladimír Zlínský
Kde: 2021ps_stenprot_139schuz_s139025.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/139schuz/s139025.htm
Kdy: 2025-05-27
Co: , že se nestala, ale stala a bylo to v roce 1859. Jmenuje se to takzvaná Carringtonova událost. A tehdy měl svět velké štěstí, protože jediné elektrické přístroje, které se používaly, v podstatě byly telegrafy, takže tehdy zcela vyhořely rozvody 
***
Kdo: Poslanec Pa

In [95]:
hledej(r"(?i)pager",sestupne=True)

In [96]:
hledej(r"(?i)magnetof",sestupne=True)

Kdo: Poslanec Patrik Nacher
Kde: 2021ps_stenprot_113schuz_s113095.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/113schuz/s113095.htm
Kdy: 2024-10-03
Co: ešli jsme od telefonu s vytáčenými čísly, s operátorem spojujícím dálkové hovory a placených telefonních budek k videohovorům s přáteli kdekoliv po světě. Postupně jsme přešli z vinylových desek přes magnetofonové pásky a CD k on-line hudbě a také od ručně psaných dopisů a telegramů k e-mailům a WhatsAppu. Jsme lidé analogového dětství a digitální dospělosti, od poslechu živých her v třílampovém rádiu a s
***
Kdo: Poslanec Tomio Okamura
Kde: 2021ps_stenprot_065schuz_s065119.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/065schuz/s065119.htm
Kdy: 2023-05-18
Co: Vážený pane premiére, podívejme se, co jste jménem ODS a koalice SPOLU říkal lidem ještě před pár týdny v televizi NOVA. (Pouští magnetofonový záznam z rozhovoru, kdy na řadu otázek, zda se budou zvyšovat daně, pan premiér odpovídá: Nebudeme zvyšovat daně. Tečka.) A teď se

In [97]:
hledej(r"(?i)disket",sestupne=True)

Kdo: Poslanec Andrej Babiš
Kde: 2021ps_stenprot_146schuz_s146012.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/146schuz/s146012.htm
Kdy: 2025-06-25
Co: e Řebíčka za ODS a Víta Bárty za Věci veřejné. Ano, Aleš Řebíček, to byl ten, co měl stavební firmu a byl ministr dopravy. Že je skvělá ODS? Berger zasáhl znovu a konstatoval ve znaleckém posudku, že diskety ze závadným obsahem nalezené u Hály mohl nastražit kdokoli a nelze prokázat, že data na disky nahrál konkrétně Hála. Takovou tezi by šlo uplatnit u jakéhokoliv digitálního nosiče, kdekoliv, tato ver
***
Kdo: Poslanec Patrik Nacher
Kde: 2021ps_stenprot_113schuz_s113095.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/113schuz/s113095.htm
Kdy: 2024-10-03
Co: u. Chodili jsme do videopůjčoven s VHS kazetami, teď se díváme na Netflix a YouTube. Pracovali jsme na sálových počítačích s děrnými štítky a domácími počítači se záznamem na magnetofonové pásky a na diskety, teď máme internet, počítače s gigabytovými paměťmi a stovky megabyt

In [98]:
hledej(r"(?i)biograf",sestupne=True)

Kdo: Poslanec David Rath
Kde: 2006ps_stenprot_032schuz_s032183.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/032schuz/s032183.htm
Kdy: 2008-06-11
Co: ým návrhem, třeba nejde k sám sobě, zamyslí se nad sebou, neřekne si vždyť já mám velké rezervy ve své práci, platí mě daňoví poplatníci, neplatí mě vůbec špatně. V době, kdy jsem provozoval soukromý biograf, tak ten mi skončil bankrotem, takže jsem rád, že mám státní dobře placené místo, tak si ho budu vážit a budu za to nějakým způsobem odvádět i určitý výkon. Jaký je výsledek? Pan poslanec Tluchoř čas
***
Kdo: Poslanec David Rath
Kde: 2006ps_stenprot_015schuz_s015048.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/015schuz/s015048.htm
Kdy: 2007-06-06
Co:  který je takto výrazný, se ještě prohloubí. To je vaše politika. To jsou důsledky vaší politiky! Dámy a pánové, každá změna má mít nějaký smysl. Mně to připadá, že vláda je v pozici člověka, který v biografu vykřikne, že hoří, a v nastalém chaosu a zmatku odcizí několika desítká

In [99]:
hledej(r"(?i)mimozemš")

Kdo: Poslanec Josef Krejsa
Kde: 1996ps_stenprot_006schuz_s006022.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/006schuz/s006022.htm
Kdy: 1996-11-01
Co: Paní předsedající, dámy a pánové, nedávno jsem zde byl svými dvěma kolegy napaden z nenávisti vůči jiným etnickým skupinám, jiným náboženstvím, jiným národům a málem i z nenávisti vůči mimozemšťanům, přestože (stenozáznamy mi dají zapravdu) jsem na půdě toto ctihodné sněmovny neřekl dodnes jediného slova proti cikánům, židům ani příslušníkům jiných ras a náboženství. Přesto jsem hrdý na to,
***
Kdo: Poslanec Miroslav Sládek
Kde: 1996ps_stenprot_008schuz_s008033.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/008schuz/s008033.htm
Kdy: 1997-02-11
Co: byla to samozřejmě trochu nadsázka, jelikož nežijeme už tolik v minulosti a hledíme do budoucna. Prohlásil jsem dnes na tiskové konferenci: ano, přiznávám se k tomu, že mohu za únos agentky Skullyové mimozemšťany. Ta logika, kterou je vedena kampaň, pomlouvačná a špinící kampaň proti re

In [100]:
hledej(r"(?i)formul\w{0,5} 1[^\w]")

Kdo: Poslanec Miloslav Kala
Kde: 2002ps_stenprot_030schuz_s030116.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/030schuz/s030116.htm
Kdy: 2004-03-26
Co: a jízdy netelefonuje, a když musí v průběhu závodu odstavit vozidlo, dělá všechno pro to, aby zaparkoval mimo možnost kontaktu s ostatními účastníky závodu. Dokonce je zajímavé, že například pravidla Formule 1 upravují situaci, kdy již se změní adhezní vlastnosti vozovky deštěm a podobně, tak vyjede takzvaný safety car, zabrzdí provoz, zakáže předjíždění a třeba i ukončí závod. Tady by stálo za úvahu, proč,
***
Kdo: Ministr průmyslu a obchodu ČR Milan Urban
Kde: 2002ps_stenprot_047schuz_s047054.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/047schuz/s047054.htm
Kdy: 2005-09-21
Co: namená to, že bychom měli být extrémní na druhou stranu, to znamená trvat na tom, když v celé Evropě bude tento proces rozhodnut, tzn. že se nepojede v Evropské unii žádný závod, ani motocyklový, ani formule 1. Na druhou stranu si myslím, že dnes ten p

In [101]:
hledej(r"(?i)dálnic")

Kdo: Místopředseda Petr
Kde: 1946uns_stenprot_083schuz_s083008.htm
URL: https://www.psp.cz/eknih/1946uns/stenprot/083schuz/s083008.htm
Kdy: 1947-11-25
Co: Žádám p. posl. Horu, aby přednesl svůj dotaz na pana ministra techniky ve věci stavby dálnice Praha - Humpolec a pražského okruhu.
***
Kdo: Posl. Hora (čte)
Kde: 1946uns_stenprot_083schuz_s083008.htm
URL: https://www.psp.cz/eknih/1946uns/stenprot/083schuz/s083008.htm
Kdy: 1947-11-25
Co: Pane předsedo, slavná sněmovno, paní a pánové! Činím stručný dotaz podle jedn. řádu na pana ministra techniky ve věci stavby dálnice Praha - Humpolec a pražského okruhu. Poloha Československé republiky může pro nás znamenat mnohé výhody soustředění komunikačních spojů a jich vedení přes naše uzemí. Od roku 1939 bylo plánováno a z
***
Kdo: K dotazu pana posl. Hory dovoluji si odpověděti takto
Kde: 1946uns_stenprot_083schuz_s083008.htm
URL: https://www.psp.cz/eknih/1946uns/stenprot/083schuz/s083008.htm
Kdy: 1947-11-25
Co: Stav, v jakém je stavba dálnice

In [102]:
hledej(r"(?i)vysokorychlostn\w{0,10} (želez|vla)")

Kdo: Místopředseda vlády a ministr dopravy ČR Milan Šimonovský
Kde: 2002ps_stenprot_048schuz_s048154.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/048schuz/s048154.htm
Kdy: 2005-10-18
Co: o úřadu o ukládání tzv. ochranných opatření v rámci ochrany trhu před nebezpečnými výrobky v oblasti součástí drah, kterou se dokončí také transpozice směrnice Rady o interoperabilitě transevropského vysokorychlostního železželez
***
Kdo: Poslanec Martin Kolovratník
Kde: 2013ps_stenprot_023schuz_s023040.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/023schuz/s023040.htm
Kdy: 2014-12-03
Co:  s vicepremiérem a ministrem financí řešili intenzivně dálnici D8 a onen zával. Co se týká železnice, máme ambiciózní plány a chystáme ve velmi krátké době rozjet přípravu vysokorychlostních spojení, vysokorychlostních železželez
***
Kdo: Poslanec Ivan Pilný
Kde: 2013ps_stenprot_023schuz_s023212.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/023schuz/s023212.htm
Kdy: 2014-12-10
Co:  výše bude známá, až se 

In [103]:
hledej(r"(?i)rychlovla")

Kdo: Posl. Biňovec
Kde: 1918ns_ps_stenprot_072schuz_s072004.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/072schuz/s072004.htm
Kdy: 1919-09-16
Co: ou dopraviti do Prahy. To si musíme říci: Vilémovi dovedli sloužit, ale naší republice sloužit nedovedou. (Hluk.) Tak to vypadá! Podívejte se do těch stanic a budete vidět! Jedete osobním vlakem nebo rychlovlakem, ale nepojedete pořádně z jedné stanice do druhé. To nejsou vinni železniční zřízenci! Úředníci ve své pasivní resistenci proti republice provádějí ty věci. Řekněte to, co nám říkáte, do svých vla
***
Kdo: Ministr železnic dr. Franke
Kde: 1918ns_ps_stenprot_096schuz_s096002.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/096schuz/s096002.htm
Kdy: 1919-12-10
Co: žen do něho tendr. Následkem této srážky byla přerušena veškerá doprava na této trati až do 12 hodin v noci, kdy vlakem č. 420 přestupováním na místě nehody byla doprava osobních vlaků zase zavedena. Rychlovlaky vedeny byly přes Vysočany-Všetaty-Lysou. Aby mohly b

In [104]:
hledej(r"VRT")

Kdo: Místopředseda PSP Tomio Okamura
Kde: 2017ps_stenprot_007schuz_s007086.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/007schuz/s007086.htm
Kdy: 2018-03-01
Co: dpověděl na interpelaci pana poslance Zbyňka Stanjury ve věci aktuálního stavu přípravy a aktuálního stavu vyhlášených výběrových řízení na zpracování studií proveditelnosti na vysokorychlostní tratě VRT v České republice. Interpelace se spolu s odpovědí předkládá jako sněmovní tisk číslo 110. Otevírám rozpravu a slovo má opět pan poslanec Zbyněk Stanjura. Pane poslanče, máte slovo.
***
Kdo: Předseda PSP Radek Vondráček
Kde: 2017ps_stenprot_012schuz_s012080.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/012schuz/s012080.htm
Kdy: 2018-04-12
Co: avy odpověděl na interpelaci poslance Zbyňka Stanjury ve věci aktuálního stavu přípravy a aktuálního stavu vyhlášených výběrových řízení na zpracování studií proveditelnosti na vysokorychlostní tratě VRT v České republice. Interpelace se spolu s odpovědí předkládá jako sněmovní ti

In [105]:
hledej(r"(?i)vzducholo",sestupne=True)

Kdo: Poslanec Richard Brabec
Kde: 2021ps_stenprot_045schuz_s045114.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/045schuz/s045114.htm
Kdy: 2022-11-30
Co:  Sněmovny, tak speciálně vy, pane ministře, protože si vás všichni pamatujeme jako velmi zdatného rétora, který jste neopomenul jakoukoliv mezírku využít, byste visel u stropu této krásné budovy jako vzducholoď Zeppelin a vyčítal byste nám to - a právem, protože si neumím představit (Potlesk poslanců Hnutí ANO.), že bych přišel jako ministr, který by měl znát každou korunu rozpočtu své kapitoly, a někde js
***
Kdo: Poslanec Michal Doktor
Kde: 2006ps_stenprot_048schuz_s048378.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/048schuz/s048378.htm
Kdy: 2009-02-20
Co: htěl bych jen zdůraznit, že to, co já absolutně podtrhuji a vítám v návrhu vlády, je fakt, že vláda si nebude vybírat ve svých opatřeních a ve svých návrzích, jestli má raději výrobce aut, letadel či vzducholodí. Já myslím, že ty veškeré akce vlády, ať jakékoliv, mají bý

In [106]:
hledej(r"(?i)[^\w]porno")

Kdo: Posl. prof. Horáček
Kde: 1918ns_ps_stenprot_094schuz_s094007.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/094schuz/s094007.htm
Kdy: 1919-12-04
Co: , já nejsem znalec, to se ale teprve ukáže. Bylo také proti mému návrhu namítáno z odborných kruhů, že tím otevřeme možnost jisté nekalé soutěži, že pak bude se prodávati nějaký literární brak, nějaké pornografické spisy a nevím jaké. Pánové, to je všecko možné, ačkoliv nevím, jestli dnešní koncesní system tomu zcela zabraňoval. Ale jest jiný obor zákonodárství, který má právě těm vadám čeliti. To jest 
***
Kdo: Posl. Němeček
Kde: 1918ns_ps_stenprot_094schuz_s094008.htm
URL: https://www.psp.cz/eknih/1918ns/ps/stenprot/094schuz/s094008.htm
Kdy: 1919-12-04
Co: oráček poukázal k tomu, že dobrý tiskový zákon bude mít úlohu, vyvarovat nejenom nebezpečí, na které bylo upozorňováno se strany interesentů knihkupeckých a knihtiskařských, totiž, aby nebyla vydávána pornografie a aby uvolněním těchto živností
***
Kdo: Posl. dr. Buday
Kde: 

In [107]:
hledej(r"(?i)počítač")

Kdo: Posl. inž. Smékal
Kde: 1960ns_stenprot_020schuz_s020003.htm
URL: https://www.psp.cz/eknih/1960ns/stenprot/020schuz/s020003.htm
Kdy: 1963-07-09
Co:  řízení výroby je nesporná. Vyšla řada pojednání, kde všude lze použít počítací stroje a přesto máme značné zpoždění v jejich používání, jak je zřejmé z toho, že např. ve Spojených státech připadá 50 počítačů na 1 mil. obyvatel, ve Švýcarsku 30 
***
Kdo: Posl. inž. Smékal
Kde: 1960ns_stenprot_023schuz_s023015.htm
URL: https://www.psp.cz/eknih/1960ns/stenprot/023schuz/s023015.htm
Kdy: 1964-01-30
Co: ozvoje a výroby výpočetní techniky. Tento úkol vytvoří především předpoklady pro zvýšení úrovně vědeckotechnických prací i pro řízení národního hospodářství. Avšak vzhledem k tomu, že vývoj středních počítačů Epos I. a Epos II. se zpozdil, nebude splněno usnesení vlády z r. 1961 pokud jsou termíny upraveny podle skutečných možností v plnění úkolů." Tolik ve zprávě. Podle mého názoru ve zprávě mělo být sp
***
Kdo: Předseda vlády Lenárt
Kde: 19

In [108]:
hledej(r"(?i)kybernet")

Kdo: Zpravodajca posl. Valo
Kde: 1954ns_stenprot_036schuz_s036006.htm
URL: https://www.psp.cz/eknih/1954ns/stenprot/036schuz/s036006.htm
Kdy: 1960-02-17
Co: díciu a skúsenosti. To je podklad, na základe ktorého je možno dosiahnuť divy. Rozvoj chémie ide vo svete míľovými krokmi vpred a myslím, že sa oprávnene hovorí, že naše storočie je storočím chémie a kybernetiky. I náš rozpočet ukazuje, aký dôležitý význam kladieme na rozvoj chemického priemyslu, kde sme práve v posledné roky zaostávali. Rozvoj strojárenstva je úzko spätý s rozvojom plastických hmôt a ché
***
Kdo: Posl. Homola
Kde: 1960ns_stenprot_004schuz_s004018.htm
URL: https://www.psp.cz/eknih/1960ns/stenprot/004schuz/s004018.htm
Kdy: 1960-11-17
Co: a mohla být uplatněna zásada "každý podle svých schopností, každému podle jeho potřeb", je nutné vytvořit vysoce výkonné průmyslové a zemědělské hospodářství s automatizací, chemizací, elektrifikací, kybernetizací, s vysokou mechanizací, je zapotřebí stále nových objevů vědy i velký

In [109]:
hledej(r"(?i)čest práci",sestupne=True)

Kdo: Poslanec Michael Rataj
Kde: 2021ps_stenprot_139schuz_s139065.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/139schuz/s139065.htm
Kdy: 2025-05-30
Co: potřebu vyváženosti. Orgány činné v trestním řízení musí odlišovat skutečné extremistické projevy od neškodných, nostalgických či ironických odkazů na minulost; například zpěv internacionály, pozdrav čest práci či nošení komunistických řádů a odznaků a podobně. Nevhodná kriminalizace by mohla paradoxně přispět k opětovné popularizaci této ideologie, a to nikdo z nás nechce. Věřím, že podobně jako v případě 
***
Kdo: Poslankyně Kristýna Zelienková
Kde: 2013ps_stenprot_031schuz_s031175.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/031schuz/s031175.htm
Kdy: 2015-09-17
Co:  logické. Takže svým komunistickým kolegům doporučuji, aby přestali trestat Ukrajince za to, že jejich komunističtí politici porušují zákon, a aby s námi dnes asociační smlouvu jednomyslně schválili. Čest práci a pojďme, prosím vás, hlasovat. Děkuji. (Potlesk nap

In [110]:
hledej(r"(?i)poslední bitva vzp",sestupne=True)

In [111]:
hledej(r"(?i)své okovy",sestupne=True)

Kdo: Ministr kultury ČR Pavel Dostál
Kde: 1998ps_stenprot_031schuz_s031031.htm
URL: https://www.psp.cz/eknih/1998ps/stenprot/031schuz/s031031.htm
Kdy: 2001-01-12
Co: Děkuji. Pane předsedající, já nebudu reagovat na slova týkající se mé rezignace. To je věcí Poslanecké sněmovny nebo pana premiéra, samozřejmě. Navíc já bych mohl jen ztratit své okovy. Chtěl bych jenom reagovat na slova pana poslance Filipa v tom smyslu, že bych chtěl bránit vládu, neboť vláda nepřijala nic v rozporu s ústavou nebo Listinou základních práv a svobod. Pan předseda F
***
Kdo: Posl. Horn
Kde: 1948ns_stenprot_057schuz_s057004.htm
URL: https://www.psp.cz/eknih/1948ns/stenprot/057schuz/s057004.htm
Kdy: 1951-07-12
Co: . Systém kapitalistické výroby právě s těmito masami vykořisťovaných tvořil sám o sobě přirozenou organisací dělnické třídy, které stačilo dát třídní vědomí toho, že proletariát nemá co ztratit, leda své okovy, získat však může všechno, třídní vědomí společenského poslání proletariátu, aby se stal p

In [112]:
hledej(r"(?i)keťas",sestupne=True)

Kdo: Poslanec Jiří Kobza
Kde: 2021ps_stenprot_108schuz_s108039.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/108schuz/s108039.htm
Kdy: 2024-06-25
Co:  naší planetu před tím, aby zamrzla. Návrh jsem podal, to ostatní dávám k zamyšlení, protože snižovat počet povolenek na trhu, aby se zvýšila jejich cena, tomuto obchodnímu způsobu se po válce říkalo keťasení a není to nic jiného než prachsprostá šmelina. Děkuji vám.
***
Kdo: Předseda vlády ČR Mirek Topolánek
Kde: 2006ps_stenprot_027schuz_s027089.htm
URL: https://www.psp.cz/eknih/2006ps/stenprot/027schuz/s027089.htm
Kdy: 2008-01-31
Co: u potravin, kde vůbec být nemusely, tak platí, že u televizních poplatků a nájmu jde o dílo vaší legislatury nebo vaší vlády, minulé Sněmovny. Konkrétně u potravin musí být zcela evidentní, že po tom keťasovském nárůstu na konci roku ceny jdou dolů, vracejí se na svoji řekl bych tržní úroveň v tržně pokrouceném prostředí, a je zcela evidentní, že vláda na to neměla absolutně žádný vliv, že cenové vlivy


In [113]:
hledej(r"(?i)privatiz[ao]")

Kdo: Ministr financí, cen a mezd Jiří Nikodým
Kde: 1986cnr_stenprot_025schuz_s025007.htm
URL: https://www.psp.cz/eknih/1986cnr/stenprot/025schuz/s025007.htm
Kdy: 1990-03-28
Co: ch v letech 1948 - 1989. Chtěl bych k tomu nejdříve říci, že na základě rozhodnutí federální vlády číslo usnesení 139 z 1. března letošního roku se zřizuje Úřad pro dočasnou správu státního majetku a privatizaci při federálním ministerstvu financí k datu 1. dubnu. Úřad má ve své činnosti postupovat podle schválené náplně práce a bude se řídit pravidly postupu při 
***
Kdo: Společný zpravodaj poslanec Josef Hrabaň
Kde: 1986cnr_stenprot_025schuz_s025011.htm
URL: https://www.psp.cz/eknih/1986cnr/stenprot/025schuz/s025011.htm
Kdy: 1990-03-29
Co: ožení, projednání a schválení rozhodujících ekonomických zákonů, které musí vytvořit základní legislativní předpoklady pro postup hospodářské reformy. Na to navazují nezbytná opatření v oblasti cen a privatizace majetku, vytvoření podmínek pro uvolnění zahraničně obchodní či

In [114]:
hledej(r"(?i)vysídl")

Kdo: Zpravodajca posl. dr Hodža
Kde: 1945pns_stenprot_048schuz_s048003.htm
URL: https://www.psp.cz/eknih/1945pns/stenprot/048schuz/s048003.htm
Kdy: 1946-04-12
Co: štát Čechov a Slovákov. Na rozdiel od spôsobu riešenia problému nemeckej menšiny, ktorý bol stanovený postupimskou konferenciou na základe princípu kolektívnej zodpovednosti celého národa, zakladá sa vysídlenie Maďarovna výmene obyvateľstva podľa Dohody medzi oboma štátmi. Hlavné zásady Dohody sú tieto: Všetky osoby národnosti slovenskej a českej, ktoré majú stále bydlisko na území Maďarska a ktoré prej
***
Kdo: Zpravodaj posl. inž. Janáček
Kde: 1945pns_stenprot_048schuz_s048005.htm
URL: https://www.psp.cz/eknih/1945pns/stenprot/048schuz/s048005.htm
Kdy: 1946-04-12
Co: éna na východním Slovensku i v zemi Moravskoslezské zpustošena byla celá území včetně lidských obydlí, továren i hospodářských a výrobních podniků, stejně jako byly postiženy četné kraje i v Čechách, vysídlené okupanty za účelem zřizování vojenských cvičišť a p

In [115]:
hledej(r"(?i)vysídlen")

Kdo: Zpravodajca posl. dr Hodža
Kde: 1945pns_stenprot_048schuz_s048003.htm
URL: https://www.psp.cz/eknih/1945pns/stenprot/048schuz/s048003.htm
Kdy: 1946-04-12
Co: štát Čechov a Slovákov. Na rozdiel od spôsobu riešenia problému nemeckej menšiny, ktorý bol stanovený postupimskou konferenciou na základe princípu kolektívnej zodpovednosti celého národa, zakladá sa vysídlenie Maďarovna výmene obyvateľstva podľa Dohody medzi oboma štátmi. Hlavné zásady Dohody sú tieto: Všetky osoby národnosti slovenskej a českej, ktoré majú stále bydlisko na území Maďarska a ktoré prejav
***
Kdo: Zpravodaj posl. inž. Janáček
Kde: 1945pns_stenprot_048schuz_s048005.htm
URL: https://www.psp.cz/eknih/1945pns/stenprot/048schuz/s048005.htm
Kdy: 1946-04-12
Co: éna na východním Slovensku i v zemi Moravskoslezské zpustošena byla celá území včetně lidských obydlí, továren i hospodářských a výrobních podniků, stejně jako byly postiženy četné kraje i v Čechách, vysídlené okupanty za účelem zřizování vojenských cvičišť a

In [116]:
hledej(r"(?i)feminis")

Kdo: Posl. Kirpalová (německy)
Kde: 1920ns_ps_stenprot_267schuz_s267002.htm
URL: https://www.psp.cz/eknih/1920ns/ps/stenprot/267schuz/s267002.htm
Kdy: 1924-06-03
Co: vná sněmovno! Hned, když se tato sněmovna sešla, bylo první prací, kterou vykonaly ženy jako poslankyně, že vytvořily zákon na ochranu matek a kojenců. Nebylo úmyslem, abychom zde chtěly konati čistě feministickou práci, nýbrž bylo to věcí samozřejmou, vyplývající z poznání, že dnes pro matky a kojence nebylo v tomto státě vykonáno nic nebo velice málo. Bohužel, tento návrh zákona byl příliš brzo znovu p
***
Kdo: Posl. Landová-Štychová
Kde: 1925ns_ps_stenprot_034schuz_s034008.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/034schuz/s034008.htm
Kdy: 1926-06-17
Co: , za to všechno je v těchto předlohách zákonů degradována na potměšilého simulanta, proti kterému se musí výjimečnými opatřeními stát brániti. A právě v tomto bodu liší se naše komunistické - nikoliv feministické - stanovisko od toho vašeho státotvornického f

In [117]:
hledej(r"(?i)elektromobil")

Kdo: Poslanec akad. B. Kvasil
Kde: 1971fs_sl_stenprot_004schuz_s004005.htm
URL: https://www.psp.cz/eknih/1971fs/sl/stenprot/004schuz/s004005.htm
Kdy: 1974-10-24
Co: arušováno životní prostředí hlukem. Proto musí být vědeckovýzkumná činnost a mezinárodní spolupráce zaměřena na taková zdokonalení vozidel, jako je snižování obrátek technické řešení pneumatik, vývoj elektromobilu, aby se zlepšily podmínky životního prostředí. Zvlášť náročným problémem v silniční dopravě je budování dálnice. Po technické stránce naše silnice budou mít parametry evropské úrovně, tj. rychlost 1
***
Kdo: Poslankyně Milada Emmerová
Kde: 2002ps_stenprot_006schuz_s006067.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/006schuz/s006067.htm
Kdy: 2002-10-09
Co: o tu několikrát diskutovali - je vlastně plzeňský Perský záliv, ale že je zde naděje, že se tohoto energetického zdroje, který působí tak nebezpečné výfukové zplodiny, zbavíme. Je tendence k zavádění elektromobilů, jsou automobily, které pracují na stlačený

In [118]:
hledej(r"(?i)Václ\w{1,5} Hav(el|la|lo|le)")

Kdo: Poslanec Václav Anděl
Kde: 1986cnr_stenprot_018schuz_s018016.htm
URL: https://www.psp.cz/eknih/1986cnr/stenprot/018schuz/s018016.htm
Kdy: 1989-12-19
Co: tejně jako téměř všichni, mám svůj díl odpovědnosti. Tuto odpovědnost mají samozřejmě i všichni členové KSČ i ti jinak poctiví, tuto odpovědnost má i většina nestraníků, ale nemají ji možná lidé typu Václava Havlala
***
Kdo: Poslanec František Adámek
Kde: 1986cnr_stenprot_021schuz_s021001.htm
URL: https://www.psp.cz/eknih/1986cnr/stenprot/021schuz/s021001.htm
Kdy: 1990-02-13
Co: i poslanci, hovořím o tom proto, že naše poslanecké mandáty rovněž žádají umění prosazovat často mnohem širší zájmy, než jen ty své stranické. Připomínám v té souvislosti slova našeho pana prezidenta Václava Havlala
***
Kdo: Ministryně obchodu a cestovního ruchu Vlasta Štěpová
Kde: 1986cnr_stenprot_021schuz_s021006.htm
URL: https://www.psp.cz/eknih/1986cnr/stenprot/021schuz/s021006.htm
Kdy: 1990-02-13
Co: ali občané důvěru k některým vedoucím osobnostem na

In [119]:
hledej(r"(?i)Adolf\w{0,5} Hitler")

Kdo: Posl. Patzel (německy)
Kde: 1920ns_ps_stenprot_299schuz_s299001.htm
URL: https://www.psp.cz/eknih/1920ns/ps/stenprot/299schuz/s299001.htm
Kdy: 1924-11-17
Co: ábřehu byl odsouzen soukromý úředník Otto Weiss okresní politickou správou podle "Prügelpatentu" k 48 hodinám vězení, poněvadž měl ve vývěsní skřínce svazu mládeže národně-socialistické vyvěšen obraz Adolfa Hitlera. Zemská politická správa zamítla odvolání s odůvodněním, že se dopustil ostentativně a provokačně rušení veřejného pokoje a řádu. K tomu jest jistě škoda každé poznámky mimo té, že pan ministr vnitr
***
Kdo: Posl. Schäfer (německy)
Kde: 1929ns_ps_stenprot_068schuz_s068004.htm
URL: https://www.psp.cz/eknih/1929ns/ps/stenprot/068schuz/s068004.htm
Kdy: 1930-09-17
Co: ebo nic" nebylo by válečným poškozencům naprosto pomoženo. Nejsou to než velká slova, která jsme slyšeli. Při tom bylo včera s tohoto místa řečeno slovo, které musí býti vytknuto. Po volebním úspěchu Adolfa Hitlera v Německu nosí i u nás němečtí národní s

In [120]:
hledej(r"(?i)Ježíš\w{0,5} Krist",sestupne=True)

Kdo: Poslanec Ivan Bartoš
Kde: 2021ps_stenprot_125schuz_s125081.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/125schuz/s125081.htm
Kdy: 2024-12-19
Co: řeba jsou nějaká projednávání důležitých legislativ, tak tady vidím pochodovat ultrakonzervativní katolíky v České republice, kteří nám tvrdí, jakým způsobem my máme žít... lidé, kteří nevěří třeba v Ježíše Krista, mají žít. My tady od rána posloucháme, že jestli se budeme k tomuto vyjadřovat, tak bude odvolána z místopředsednického postu Olga Richterová. Tohle musím naprosto odmítnout! Co to je za chování? T
***
Kdo: Poslanec Miloslav Janulík
Kde: 2021ps_stenprot_124schuz_s124084.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/124schuz/s124084.htm
Kdy: 2024-12-11
Co: lastně slouží? Když to neslouží lidu této země, tak komu to teda slouží? Tady se vždycky kolega Brázdil ptá - proč to vlastně všecko je? Komu vadí pravda jako taková? To už asi od dob Husa nebo možná Ježíše Krista. Kdo se toho bojí, že si řekne, jak ty věci jsou? A úp

In [121]:
hledej(r"(?i)televiz",sestupne=False)

Kdo: Posl. Houser
Kde: 1954ns_stenprot_017schuz_s017017.htm
URL: https://www.psp.cz/eknih/1954ns/stenprot/017schuz/s017017.htm
Kdy: 1957-04-17
Co: tí přinese do roku 1960 úspory 27 mil. Kčs. Jeden z nejmladších závodů našeho kraje, Rožnovskou Teslu, čekají zvýšené úkoly ve výrobě ještě kvalitnějších elektronek, hlavně snímacích a obrazových pro televizní přístroje. Dosavadní přípravy nasvědčují, že náš spotřebitelský trh bude zásoben takovými 
***
Kdo: Posl. Drda
Kde: 1954ns_stenprot_024schuz_s024024.htm
URL: https://www.psp.cz/eknih/1954ns/stenprot/024schuz/s024024.htm
Kdy: 1958-04-16
Co: árných pramenů. Osvobození sociální má jako důsledek neustále vzrůstající kulturní dychtivost. Máme tolik divadel, jako nikde na světě, pokud běží o hustotu obyvatelstva, takovou hustotu rozhlasové a televizní sítě, tolik čtenářů, diváků a posluchačů, že to budí ve světě podiv. Pro socialistickou výchovu lidu jsme vytvořili tolik nástrojů a prostředků, že prakticky každý náš občan každodenně s někter

In [122]:
hledej(r"(?i)rozhlas[^\w]",sestupne=False)

Kdo: Posl. inž. Hrdina
Kde: 1925ns_ps_stenprot_052schuz_s052015.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/052schuz/s052015.htm
Kdy: 1926-11-24
Co: to již na školách středních i vysokých a zvláště na ústavech učitelských. Lidové výchově přibylo úkolů velikými pokroky technickými, jichž dobrodiní je nutno využíti na prospěch venkova: jsou to radiorozhlas a film. Ministerstvo školství a nár. osvěty je na svém místě k tomu povoláno, aby obou uvedených prostředků využilo ve službách lidové výchovy. Chceme-li, aby film sloužil více kultuře občanstva než d
***
Kdo: Posl. Kvasnička
Kde: 1925ns_ps_stenprot_111schuz_s111016.htm
URL: https://www.psp.cz/eknih/1925ns/ps/stenprot/111schuz/s111016.htm
Kdy: 1927-12-01
Co: nictvím radia. (Výborně!) Otázka finanční nemůže býti rozhodující - samo ministerstvo o tom mluvilo - poněvadž příjmy z poplatků jsou tak velké a skvělé, že tento náklad nepřichází vůbec v úvahu. Ani rozhlas v Praze a v Brně neuspokojuje. Dnešní stanice bude nutno nahradit

In [123]:
hledej(r"(?i)pětiden\w{1,5} pracovn",sestupne=False)

Kdo: Posl. Hruška
Kde: 1929ns_ps_stenprot_037schuz_s037006.htm
URL: https://www.psp.cz/eknih/1929ns/ps/stenprot/037schuz/s037006.htm
Kdy: 1930-04-03
Co: emědělce a likvidace kulaků. Stejných úspěchů jako na poli hospodářském dosahuje sovětský Svaz také na poli kulturním. Povznesení sociální úrovně širokých pracujících mas, sedmihodinná doba pracovní, pětidenní pracovní týden, zvýšení mezd, zdokonalení sociálních zařízení, úspěšný boj s negramotností a kulturní zaostalostí povznášejí netušeně kulturní úroveň pracujícího lidu měst i venkova. Likvidace kulaků a vítěz
***
Kdo: Posl. K. Procházka
Kde: 1929ns_ps_stenprot_045schuz_s045006.htm
URL: https://www.psp.cz/eknih/1929ns/ps/stenprot/045schuz/s045006.htm
Kdy: 1930-05-02
Co: starají, aby lidé práce schopní do výroby byli skutečně zařazeni. Kdybyste zde na místo zákonů, které nemají jiného účelu než podváděti dělnictvo, uzákonili 7hodinovou pracovní dobu bez snížení mezd, pětidenní pracovní týden bez újmy na výdělku, bezplatné pojištění 

In [124]:
hledej(r"(?i)důchod\w{1,5} reform",sestupne=False)

Kdo: Poslanec Jaroslav Štrait
Kde: 1993ps_stenprot_016schuz_s016006.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/016schuz/s016006.htm
Kdy: 1994-02-16
Co: vládou, odbory, zaměstnavateli a pojišťovnami. 5. Předpokládaná 40 %ní zapojenost populace produktivního věku během tří let není ekonomicky podložena, má spekulativní charakter a sociální průchodnost důchodové reformy negarantuje. Dovolte mi z výše uvedených důvodů projednávání bodu 13 přerušit a pokračovat v jeho projednávání na zasedání Poslanecké sněmovny tam, kde budeme projednávat současně zákon o základním 
***
Kdo: Poslanec Pavel Šafařík
Kde: 1993ps_stenprot_032schuz_s032057.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/032schuz/s032057.htm
Kdy: 1995-06-29
Co: reformě mluví jistě s nadsázkou. Dovolil bych si s celkovým pojetím a koncepčním záměrem navrhovatelů polemizovat. Nechci vás unavovat detailním popisem směrů a záměrů, kterými se postoje realizátorů důchodové reformy pohybovaly. Pocit potřebné a skutečné reformy

In [125]:
hledej(r"(?i)církevn\w{1,5} restituc",sestupne=False)

Kdo: Poslanec Zdeněk Vorlíček
Kde: 1993ps_stenprot_013schuz_s013006.htm
URL: https://www.psp.cz/eknih/1993ps/stenprot/013schuz/s013006.htm
Kdy: 1993-10-12
Co: y občanů v této oblasti a dlouhodobě nevěnovala dostatek pozornosti a času potřebným změnám v trestním zákonodárství a zesílení trestní represe. Vždycky bylo něco důležitější. Dneska všechno ovládají církevní restituce. Co nás tato laxnost při řešení bezpečnostní situace stojí zahraničně politicky a ekonomicky, nám sdělil v hospodářském výboru nedávno ředitel Czechinvestu, když komentoval situaci v zahraničí poté,
***
Kdo: Poslanec Pavel Dostál
Kde: 1996ps_stenprot_003schuz_s003005.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/003schuz/s003005.htm
Kdy: 1996-07-24
Co: en, protože vláda hodlá tento převod řešit bez zákona, exekutivní cestou. Jinými slovy, nikdo z nás už v podstatě nebude rozhodovat o tom. zda tento majetek vydán bude. To, prosím vás, není nic proti církevním restitucím. Možná bych měl na celou věc úplně jiný po

In [126]:
hledej(r"(?i)kosmonaut",sestupne=False)

Kdo: Předseda
Kde: 1960ns_stenprot_007schuz_s007001.htm
URL: https://www.psp.cz/eknih/1960ns/stenprot/007schuz/s007001.htm
Kdy: 1961-04-17
Co: tická země Sovětský svaz, jejíž zřízení dokázalo uvolnit největší tvůrčí síly člověka a krok za krokem naplňovat jeho nejsmělejší cíle. Vypuštění kosmické sovětské lodi s prvním člověkem se sovětským kosmonautem Jurijem Gagarinem a jeho šťastný návrat na zem je symbolem mírového humanistického úsilí sovětské vědy a Sovětského svazu a je současně novým skvělým důkazem o převaze socialistické vědy i socialis
***
Kdo: Předseda NS Fierlinger
Kde: 1960ns_stenprot_015schuz_s015001.htm
URL: https://www.psp.cz/eknih/1960ns/stenprot/015schuz/s015001.htm
Kdy: 1962-07-05
Co: ovu co největšího počtu odborných a vědeckých pracovníků. O tom jsme mluvili zejména při návštěvě Lomonosovské university na Leninských gorách. O úspěších sovětské vědy nejlépe svědčí lety sovětských kosmonautů do vesmíru a četné jiné velké objevné činy sovětských vědců ve všech vědníc

In [127]:
hledej(r"(?i)astronaut",sestupne=False)

Kdo: Posl. Vecker
Kde: 1954ns_stenprot_036schuz_s036005.htm
URL: https://www.psp.cz/eknih/1954ns/stenprot/036schuz/s036005.htm
Kdy: 1960-02-17
Co:  proto rozhořčeně odsuzuje ty, kteří proti vůli národů chtějí zůstávat na pozicích studené války. Mladí lidé dychtivě sledují veliké objevy vědy a techniky, jsou vzrušeni představou o cestách prvních astronautů. A taková je mládež v celém světě. Přesto, že v kapitalistickém Západě vychovávají mládež k pesimismu a beznaději, zabijáctví a cynismu, poznali jsme mnoho mladých lidí, jimž imponuje nikoliv výbuch
***
Kdo: Kernerová
Kde: 1960ns_stenprot_005schuz_s005002.htm
URL: https://www.psp.cz/eknih/1960ns/stenprot/005schuz/s005002.htm
Kdy: 1960-12-15
Co: ích metodicky řídit, jednotně plánovat a ústředně koordinovat. V Sovětském svazu je tato forma studia již běžná a úspěchy sovětské vědy a techniky ve všech oborech, prvenství Sovětského svazu např. v astronautice, jsou jen dokladem toho, jak veliké možnosti růstu úrovně techniky a věd péče o st

In [128]:
hledej(r"(?i)Jan\w{0,5} Hus",sestupne=True)

Kdo: Poslanec Jan Síla
Kde: 2021ps_stenprot_131schuz_s131109.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/131schuz/s131109.htm
Kdy: 2025-03-06
Co: d ní se odštěpila pouze husitská církev, která byla tolerovaná a vnímaná jako dobrá. Letos je tomu sto let od výročí Marmaggiho aféry, po zrušení tří mariánských svátků a naopak vyhlášení Dne upálení Jana Husa. Oslavy v roce 1925 se účastnila nejen vláda, ale i Masaryk jako prezident Masaryk, což římskokatolická církev považovala za provokaci, a nuncius Marmaggi opustil Českou republiku z jednoho prostého
***
Kdo: Poslanec Tomio Okamura
Kde: 2021ps_stenprot_129schuz_s129018.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/129schuz/s129018.htm
Kdy: 2025-02-12
Co: , že můžu bez problému říkat, notabene když je to ještě v souladu s programem, za který je SPD zvoleno, ale já jsem jí chtěl říct jednu věc a nakonec jsem v hlavě nad tím mávnul rukou. No, on i mistr Jan Hus mohl mluvit před tím soudem, před tou, řekl bych, inkvizicí. Ona i Mil

In [267]:
hledej(r"(?i)manželstv\w{0,10} pro vše")

Kdo: Poslankyně Vlasta Parkanová
Kde: 2002ps_stenprot_035schuz_s035090.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/035schuz/s035090.htm
Kdy: 2004-09-22
Co: o první, dnes můžeme vidět, že dalším stupněm po uzákonění registrovaného partnerství je zavádění pohlavně neutrálního manželství, tedy žádné dělení na homosexuální a heterosexuální svazky, ale jedno manželství pro všechny bez rozdílu. Takový požadavek je zcela v souladu s logikou "zrovnoprávnění" homosexuálních svazků, a pokud nastoupíme cestu, kterou otevírá předložený návrh zákona, těžko můžeme dříve či později 
***
Kdo: Poslanec František Kopřiva
Kde: 2017ps_stenprot_020schuz_s020389.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/020schuz/s020389.htm
Kdy: 2018-11-14
Co: stně k tomu, co je předmětem návrhu, změní se pouze pár slov v občanském zákoníku. Dále, ve Velké Británii se snížení počtu sebevražd mladých lidí někdy dává do souvislosti právě s uzákoněním rovného manželství pro všechny. Co se týče opět nějakých tradi

In [130]:
hledej(r"(?i)no[- ]go zón")

Kdo: Poslanec Marek Černoch
Kde: 2013ps_stenprot_030schuz_s030033.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/030schuz/s030033.htm
Kdy: 2015-10-01
Co: podíváme na ty obrázky, které přicházejí z Maďarska, obrázky, které přicházejí z Francie, z Německa, tak si myslím, že nikdo z nás nechce, aby se podobné věci děly i u nás v České republice. Nechceme no-go zóny, nechceme sociální problémy, nechceme ani, aby se opravdu situace pak vyvíjela absolutně radikálním směrem, kdy dnes celkově v Evropě je spíše otázkou času, kdy lidé vezmou věci do svých vlastních r
***
Kdo: Poslanec Martin Lank
Kde: 2013ps_stenprot_031schuz_s031290.htm
URL: https://www.psp.cz/eknih/2013ps/stenprot/031schuz/s031290.htm
Kdy: 2015-10-02
Co: okové zvýšení kriminality. Já jsem si stoprocentně jistý, že nelegálním imigrantům, kteří porušují zákony České republiky, azyl dávat nesmíme. Pak se můžeme bavit samozřejmě ještě o vzniku takzvaných no-go zón. Ty existují po celé západní Evropě, přestal tam v podstatě plat

In [131]:
hledej(r"(?i)vznášedl")

Kdo: Poslanec Josef Krejsa
Kde: 1996ps_stenprot_020schuz_s020362.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/020schuz/s020362.htm
Kdy: 1998-02-25
Co: al, pardon, kolega Dostál. Možná právem, možná mylně předpokládám, že nejsem sám, komu se hloubkový vrt kolegy do studnice našeho poznání ne zcela vydařil. Abych posadil jeho levotočivé intelektuální vznášedlo se vznětovým uměleckým pohonem na mísu šedé reality našeho ctihodného sboru a otevřel jeho předložený návrh registrovatelným vjemům průměrného poslance tohoto parlamentu se 14procentní důvěrou občan
***
Kdo: Poslanec Daniel Korte
Kde: 2010ps_stenprot_049schuz_s049022.htm
URL: https://www.psp.cz/eknih/2010ps/stenprot/049schuz/s049022.htm
Kdy: 2012-12-04
Co: ovat botičky, když nějaký nebožák zastaví v parkovací zóně, aby si skočil do trafiky pro cigarety. Dámy a pánové, městský strážník má šlapat chodník s píšťalkou v ruce, a ne se prohánět v hummerech a vznášedlech po městě! Je to ovšem vládní návrh, a já jsem proto se skřípají

In [132]:
hledej(r"(?i)sexu\w{0,10} výchov")

Kdo: Ministr školství ČSR doc. Milan Vondruška
Kde: 1981cnr_stenprot_012schuz_s012002.htm
URL: https://www.psp.cz/eknih/1981cnr/stenprot/012schuz/s012002.htm
Kdy: 1984-04-02
Co:  oborů. Na některých vysokých školách se vyučuje nejen biotechnologiím, elektronice, kybernetice, automatizaci apod., ale zároveň se začíná fakultativní výuka estetiky, morálky, sociální psychologie, sexuální výchovy apod. Prohlubuje se i spolupráce se zeměmi socialistického společenství při výchově vysokoškolských odborníků. Vláda ČSR a ministerstvo školství jsou si vědomy, že některým absolventům vysokých škol
***
Kdo: Poslankyně Marcela Černá
Kde: 1986cnr_stenprot_003schuz_s003007.htm
URL: https://www.psp.cz/eknih/1986cnr/stenprot/003schuz/s003007.htm
Kdy: 1986-10-20
Co: mise, těhotenství nechtěla a neodborně situaci řešila se všemi následky. Nový zákon také umožní rozšířit bezplatné používání antikoncepce. Profesionálně se my, gynekologové, nezříkáme svého podílu na sexuální výchově a na výchově k plánované

In [133]:
hledej(r"(?i)uměl[^e]\w{0,10} intelig")

Kdo: Předseda vlády ČR Andrej Babiš
Kde: 2017ps_stenprot_023schuz_s023006.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/023schuz/s023006.htm
Kdy: 2018-11-23
Co: ci, kdy si skutečně můžeme vybírat. Je jasná naše ekonomická strategie. Věda a výzkum a finální výroba. V Bruselu u předsedy Evropské komise jsem lobboval za to, abychom získali centrum excelence pro umělou inteligenci. To je pro nás strašně důležité. Novelu zákona o nemocenském pojištění zavedeme od července příštího roku. Zavedeme elektronickou neschopenku, další důležitý krok k elektronizaci zdravotnictví. T
***
Kdo: Předseda vlády ČR Andrej Babiš
Kde: 2017ps_stenprot_026schuz_s026245.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/026schuz/s026245.htm
Kdy: 2019-01-31
Co: kotvení digitalizace; v prosinci 2018 představení Sonia, bankovní identita pro státní správu, po realizaci získáme kmen 4 milionů klientů do státní správy; v prosinci 2018 schválena strategie využití umělé inteligence v rámci našeho programu Česká rep

In [134]:
hledej(r"(?i)legaliz\w{0,10} marih")

Kdo: Poslanec Jindřich Nehera
Kde: 1996ps_stenprot_012schuz_s012052.htm
URL: https://www.psp.cz/eknih/1996ps/stenprot/012schuz/s012052.htm
Kdy: 1997-07-08
Co:  a dalších 9 % jich zná více než dva. Na otázku, zda je z dotazovaných pro zvýšení trestu sazby pro distributory drog, odpovědělo 83,6 % kladně, 12,7 % záporně a nezaujímají stanovisko pouze 2 %. Pro legalizaci marihuany se vyjádřilo 20 % dotázaných, proti je 74,5 % dotázaných a neví 3,5 % dotázaných. 74,6 % se považuje za informované o problematice drog, 12,7 % jen za částečně informované a 12,7 % za neinformova
***
Kdo: Poslanec Boris Šťastný
Kde: 2010ps_stenprot_040schuz_s040153.htm
URL: https://www.psp.cz/eknih/2010ps/stenprot/040schuz/s040153.htm
Kdy: 2012-06-08
Co:  návrhu jakkoli marihuanu zneužít a dostat ji mimo ten systém, nebo minimalizovat rizika, že se dostane mimo systém léčby. To je správné, a kdo by tento návrh zavrhoval proto, že bude říkat, že to je legalizace marihuany, tak to je absolutní nesmysl. Tak to není.